# Expected goal model using logistic regression

Welcome back to another tutorial about soccer analytics. While in the previous notebooks it was (almost) all about visualizing data, in this notebook we will for the first time look into machine learning and jointly build a model to predict expected goals. If you haven't heard about expected goals, I recommend you to watch this great [video](https://www.youtube.com/watch?v=Xc6IG9-Dt18). 

In this notebook we will not only look at how to train a logistic regression, but also touch a lot of other steps required in the development of a machine-learning model such as finding an appropriate metric, feature engineering and feature interpretation. So, if you are interested in getting started in machine learning, I think this can be a great starting point (even though I admit that this notebook is rather long and it will probably take you quite some time to go through it :-)). I promise that you will be rewarded with not only a very decent expected goal model but you will have (hopefully) understood how these things work under the hood and be able to build another model yourself. 

The notebook starts with a bad (but unfortunately not completely uncommon) modelling approach that we improve over the remainder of the notebook. It is split into the following sections:
0. [First (shitty) model](#shitty_model) <br>We build a first model for which we outline the errors made over the course of this notebook.


1. [Bias detection](#bias) <br>We will look at the underlying data and understand why they convey an inherit bias.


2. [Metric definition](#metric) <br>We define an appropriate metric for our problem and talk about the problems other metrics might have.


3. [Treating outliers](#outlier) <br>We look into outliers and how we can deal with them.


4. [Univariate analysis](#univariate) <br>We engineer a first set of features to be usable in our logistic regression approach.


5. [Feature creation](#features) <br>We create additional features and learn how to get a good feeling on whether they might be useful.


6. [Multivariate analysis](#multivariate) <br>We look at the complete set of features and their correlations.


7. [Final model](#final) <br>We build a final model and check for its performance


8. [Feature interpretation](#feature_interpretation) <br>We learn how to interpret features in a logistic regression


9. [Model usage](#model_usage) <br>We quickly talk about why always having the use case in mind is crucial for developing a good model. 

We will go into all the topics in some detail but I really encourage you to read more on these topics and play around with it while going through the notebook! 

So, let's get started! I hope you enjoy the notebook and learn something along the way! :-)

In [56]:
# import packages
import os
import pandas as pd
import numpy as np
import plotly.express as px
import math

import sklearn.metrics as sk_metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from statsmodels.stats.outliers_influence import variance_inflation_factor    

if os.getcwd().split(os.sep)[-1] == "notebooks":
    os.chdir("../")

import helper.io as io
import helper.plotly as py_help
import helper.machine_learning as ml_help
import helper.expected_goal_model as goal_model
import helper.event_data as ed_help

# this is very useful as it makes sure that always all columns and rows of a data frame are displayed
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Read the event data of all the leagues

In [57]:
league = "all"
df_events = io.read_event_data(league, notebook="expected_goal_model")

In [58]:
df_events.head()

,id,matchId,matchPeriod,eventSec,eventName,subEventName,teamId,posBeforeXMeters,posBeforeYMeters,posAfterXMeters,posAfterYMeters,playerId,playerName,playerPosition,playerStrongFoot,teamPossession,homeTeamId,awayTeamId,goal,ownGoal,counterAttack,bodyPartShot,bodyPartShotCode
0,179896442,2516739,1H,2.409746,Pass,Simple pass,2446,52.5,34.00,52.50,32.64,15231,K. Volland,FW,left,2446,2444,2446,0,0,0,Unknown,0
1,179896443,2516739,1H,2.506082,Pass,Simple pass,2446,52.5,32.64,23.10,14.96,14786,K. Bellarabi,MD,right,2446,2444,2446,0,0,0,Unknown,0
2,179896444,2516739,1H,6.946706,Pass,Simple pass,2446,23.1,14.96,6.30,31.28,14803,S. Bender,DF,right,2446,2444,2446,0,0,0,Unknown,0
3,179896445,2516739,1H,10.786491,Pass,Simple pass,2446,6.3,31.28,21.00,6.80,14768,B. Leno,GK,right,2446,2444,2446,0,0,0,Unknown,0
4,179896446,2516739,1H,12.684514,Pass,Simple pass,2446,21.0,6.80,28.35,2.72,14803,S. Bender,DF,right,2446,2444,2446,0,0,0,Unknown,0


Extract only the shots as we are currently only interested in those

In [59]:
df_shots = df_events[df_events["eventName"] == "Shot"].copy()

In [60]:
df_shots.head()

,id,matchId,matchPeriod,eventSec,eventName,subEventName,teamId,posBeforeXMeters,posBeforeYMeters,posAfterXMeters,posAfterYMeters,playerId,playerName,playerPosition,playerStrongFoot,teamPossession,homeTeamId,awayTeamId,goal,ownGoal,counterAttack,bodyPartShot,bodyPartShotCode
104,179896573,2516739,1H,247.703507,Shot,Shot,2444,87.15,44.88,0.0,0.0,209091,C. Tolisso,MD,right,2444,2444,2446,0,0,0,rightFoot,3
178,179896639,2516739,1H,529.393731,Shot,Shot,2444,99.75,40.12,0.0,0.0,134383,N. Süle,DF,right,2444,2444,2446,1,0,0,head/body,2
216,179896684,2516739,1H,668.234340,Shot,Shot,2446,95.55,44.88,105.0,68.0,105619,A. Mehmedi,FW,right,2446,2444,2446,0,0,0,rightFoot,3
220,179896693,2516739,1H,672.925920,Shot,Shot,2446,92.40,33.32,105.0,68.0,14786,K. Bellarabi,MD,right,2446,2444,2446,0,0,0,rightFoot,3
313,179896798,2516739,1H,949.131592,Shot,Shot,2444,77.70,28.56,0.0,0.0,20475,A. Vidal,MD,right,2444,2444,2446,0,0,0,rightFoot,3


Ok, we can directly see 3 variables that might be interesting for our expected goal model: 

1. The position of the shot stored in *posBeforeXMeters* and *posBeforeYMeters*
2. The body part the shot was taken with, i.e. left foot, right foot or head/body. This information is already encoded in *bodyPartShotCode*
3. Whether or not the shot happened after a *counterAttack*

But before we start with the modelling, let's quickly get an overview over how many shots we have and how many goals resulted out of those shots.

In [61]:
total_nb_shots = len(df_shots)
print(f"Total shots: {total_nb_shots}")
print(f"Expected goal when shooting: {df_shots['goal'].mean()*100:.2f}%")

Total shots: 40461
Expected goal when shooting: 10.56%


<a id="shitty_model"></a>

# Building a first (shitty) model :-)

Let's build a first logistic regression model. <b>When you go through the next couple lines of code, please do not stop after this section! On purpose, I'm doing basically everything wrong that can be done wrong.</b> The reason I'm doing this, is to afterwards show you how things can be done better and hopefully help you to avoid these kind of mistakes in the future (even though some of them might look made up, I have seen all of them in real-life...) So let's start to build our (shitty :-)) model.

We first extract the variables that we want to use; namely the position of the shot, the body part (i.e. left foot, right foot, head/body) the shot was taken with as well as whether the shot happened after a counter attack

In [62]:
feature_cols = ["posBeforeXMeters", "posBeforeYMeters", "bodyPartShotCode", "counterAttack"]
target_col = ["goal"]

X = df_shots[feature_cols]
y = df_shots[target_col]

Let's split the observations into training and test set

In [63]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

Train a logistic regression on the training set...

In [64]:
reg_shitty = LogisticRegression(random_state=42)
reg_shitty.fit(X_train, np.array(y_train).ravel())

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

... and predict on the test set

In [65]:
pred_vals = reg_shitty.predict(X_test)

Let's see what kind of accuracy with have with the model

In [66]:
print(f"Accuracy of the model is {sk_metrics.accuracy_score(y_test, pred_vals)*100:.1f}%")

Accuracy of the model is 89.4%


Wow, we are right in almost 90% of the cases! Not so bad, is it? 

As logistic regression is a linear model, we can easily get some interpretation out of it by looking at the coefficients

In [67]:
for i, col in enumerate(X_train.columns):
    print(f"Coefficient of {col}: {reg_shitty.coef_[0][i]:.3f}")

Coefficient of posBeforeXMeters: 0.127
Coefficient of posBeforeYMeters: -0.002
Coefficient of bodyPartShotCode: 0.022
Coefficient of counterAttack: 0.446


So what do we have: 
1. *posBeforeXMeters* is measured as distance from the attacker's goal. Meaning, the bigger this number the closer the attacker is to the opponent's goal. And the chance of actually making the goal raises by 12.7% with every meter the attacker is closer to the opponent's goal
2. After a counter attack there is a 44.6% higher chance of scoring
3. The body part does not seem to have any big effect on whether or not an attacker scores

Hm, does not sound completely weird. So let's take those results and communicate them to the coach. Also, let's see if it makes sense to establish a new tactics: "Always shoot whenever we have the ball in the box." Fortunately, we can easily measure if this tactics will work out by just comparing the current goals with the predicted number of goals when applying the tactics. :-)

Again, <b><font color='red'> please do not stop reading here </font></b>. 

When going through the code above, did you already notice some mistakes that were made? From my perspective the main errors were: 

1. Ignoring bias in the underlying data
2. Inappropriate metric to measure model performance
3. No outlier treatment
4. No focus on meaningful features, feature transformation & dummy encoding
5. No creation of new features from existing data
6. No multivariate analysis
7. Wrong interpretation of coefficients 

So let's go through the list and see if we can improve things to end up with a good expected goal model using logistic regression.

<a id="bias"></a>

# 1. Bias in the underlying data

Before even starting the model building process, it is crucial to think about what bias the underlying data might have and how this affects the questions you are able to answer with the data at hand. What do I mean with this? 

Let's start with the bias in the data: Our data source consists of all the shots that were taken in one of the 5 big European leagues in the 2017/2018 season and whether or not those shots resulted in a goal. You see the bias? Think about in which case there is a data point generated: A player takes a shot when
- he is close enough to the goal that he thinks he can make it
- there is most likely no defender right in front of him
- there is most likely no better positioned player

To put that more general: Fairly good players (and I assume all those professionals are fairly good as they play in the first league) only shoot when they do not see any better option which would result in a higher probability of scoring. Or put another way: In the data you will most likely not see (many) shots that were taken in a situation in which it was completely stupid to shoot. You see the bias?!

Ok, understood, so what kind of impact does that have on the analysis we are able to make with our expected goal model? 

<b> 1. Identification the team that should have won </b>

This is arguably the most common use case of the expected goal model. One calculates the expected goals for each team and compares them. The team that has more expected goals is the one that should have won or least drawn the match. While this analysis also has some flaws such as the fact, that good chances that did not end in a shot, are not counted for, it has proven to a be an interesting and valuable KPI. 

<b> 2. Simulation of the new "shoot whenever in the box" tactics </b>

You probably already know the answer. This is an analysis we can definitely not do with our model. Why? Shooting whenever we have the ball in the box would result in also taking stupid shots, i.e. shots with a defender right in front of us or headers from 15 meters. Those occurences, however, we do not have in our data (or at least very rarely) and can therefore also not learn from them. Worse, however, our model "automatically" assumes that all the shots taken are somehow reasonable and therefore also assumes that the shots we take with our new tactics are reasonable and with no defender right in front. This would result in a severe overestimation of the expected goals and in the worst case we would install this new tactics in a match as the simulation has shown it makes sense. Ok, nobody would actually do that, but I hope you get the point? :-)

<a id="metric"></a>

# 2. Finding a good metric

In order to validate model performance you need to measure it through an appropriate metric. Let us quickly think about what we want to have from our model... We want the expected goal model to return the <b>probability</b> of scoring a goal given that a player takes a shot under certain circumstances (position of the shot, counter attack, left foot, ...) As we are interested in the probability we should try to find a metric that measures how good our <b>probability forecast</b> is! So what did we do wrong? 

#### Accuracy

Accuracy measures the share of right and wrong decisions. That's it. It does not take into consideration at all how sure you are about the decision. It does not matter whether you are 51% sure or 99% sure. And even worse, when having unbalanced data, i.e. on outcome happens way more often than the other, it is very simple to get a good accuracy. All you need to do is to just always predict the outcome that is more likely. And guess what our model did...

In [68]:
np.unique(pred_vals)

array([0])

Exactly, it did always predict that a shot is no goal! Don't get me wrong, I'm not saying that it is bad to always predict "no goal". All I'm saying is that accuracy is definitely not a good measure to evaluate the model performance for our problem. Honestly, there is basically almost never a situation where you should use accuracy as the metric. But this is a different story...

#### Log loss

As a rule of thumb, when predicting probabilities the metric you should always look at is the log loss (also called binary cross entropy). I'm not going to talk about what it exactly is, but you have some nice article [here](https://towardsdatascience.com/understanding-binary-cross-entropy-log-loss-a-visual-explanation-a3ac6025181a).

So let's compute the log loss of our model. We first need to get the predicted probabilites from the model

In [69]:
pred_probs = reg_shitty.predict_proba(X_test)[:,1]
print(f"Log loss of our model: {sk_metrics.log_loss(y_test, pred_probs):.5f}")

Log loss of our model: 0.30553


We have a log loss of 0.30553. Hm, not completely sure if this is any good. So for comparison, let's compute a model that just always predicts 10.56%, i.e. the average expected goal over all shots taken (see above)

In [70]:
print(f"Log loss of dummy: {sk_metrics.log_loss(y_test, [0.1056]*len(y_test)):.3f}")

Log loss of dummy: 0.338


Ok, gladly, we are at least slightly better than just always predicting the average. However, to get a good understanding of how good we really are, we will need to compare the ~0.306 to other models

#### AUC

There is one more metric I would like to quickly talk about as it is very commonly (and often wrongly) used in binary classification problems. The metric I'm talking about is called the ROC AUC or just AUC. If you quickly search for it you will find a ton of websites explaining what the AUC is and how it is constructed using false positive rates and true positive rates with different thresholds etc. While those websites are obviously right in their explanation, I still find it very difficult to grasp what the AUC really means... Fortunately, there is a very nice interpretation of the AUC that most often is not mentioned when talking about the AUC (don't ask me why ;-)): 

*The AUC of a model is the probability that the model ranks a random positive example (i.e. a successful shot) more highly than a random negative example (i.e. unsuccessful shot)*

That's a nice interpretation, isn't it? So what's the problem with measuring our expected goal model performance through the AUC? If you read the interpretation again, you might notice, that it only talks about the ranking of the positive vs. the negative examples and not about the probabilities that are assigned to the examples. We, however, are interested in knowing the <b>probability</b> of a shot being successful. And this is not captured by the AUC at all... Let's make this clearer with an example:

In [71]:
# compute the AUC of our shitty model
print(f"AUC of our model: {sk_metrics.roc_auc_score(y_test, pred_probs)*100:.2f}%")

AUC of our model: 73.32%


Ok, so that means that when randomly selecting a successful shot and an unsuccessful shot, there is a 73.32% chance that our model gives the successful shot a higher probability than the unsuccessful shot. 

Let's now make our shitty model even shittier by completely overestimating the probabilites of a ball going in

In [72]:
print(f"Currently predicted success when shooting: {np.mean(pred_probs)*100:.2f}%")

# completely overestimate the probabilities
pred_probs_over = pred_probs * 6
print(f"Overestimated predicted success when shooting: {np.mean(pred_probs_over)*100:.2f}%")

Currently predicted success when shooting: 10.51%
Overestimated predicted success when shooting: 63.08%


Now we predict that 63% of all shots are successful. Something that is completely ridiculous. But guess what happens when we look at the two metrics *log loss* and *AUC*

In [73]:
print(f"Log loss w/o overestimation: {sk_metrics.log_loss(y_test, pred_probs):.3f}")
print(f"Log loss with overestimation: {sk_metrics.log_loss(y_test, pred_probs_over):.3f}")
print(f"AUC w/o overestimation: {sk_metrics.roc_auc_score(y_test, pred_probs)*100:.2f}%")
print(f"AUC with overestimation: {sk_metrics.roc_auc_score(y_test, pred_probs_over)*100:.2f}%")

Log loss w/o overestimation: 0.306
Log loss with overestimation: 6.456
AUC w/o overestimation: 73.32%
AUC with overestimation: 73.32%


Wow! While the log loss skyrocketed as expected, the AUC remained exactly the same... Again, this is because the AUC only measures the ranking between the shots, but does not at all measure if the probabilities make any sense. You see why you should not rely on the AUC when evaluating your expected goal model?

So, is the AUC completely useless in our case? Some people might argue that yes, but in my opinion it can still serve one very important task: It can help you to communicate your results to non data scientists. So even though you focus on improving the log loss during model development, in case someone comes by and asks you how good your model is, you might rather answer "*It has a 73% chance of detecting the successful shot from the unsuccessful one*" than "*It has a log loss of 0.306*". You see what I mean :-)

Nice, so we have already learned that our data has some bias and we know which metric we want to use to evaluate our model. Let's go to the next step, the outlier removal.

<a id="outlier"></a>

# 3. Outlier treatment

One very important step before building any model is to check, that the incoming data is accurate and treat potential outliers. And I really urge you to do this carefully as even a small number of outliers can have a huge impact on your final result...

So what is an outlier? In my opinion outliers include two types of data points:

1. Data points that are wrong, i.e. shots that never happened
2. Data points that are correct, i.e. shot happened in reality, but you do not want the model to learn from

Ok, I guess the first point is clear. If it a shot is just wrongly measured and never happened in reality, we definitely need to do something about it. 

But what about the second point, what do I mean with that? Imagine the following situation: It is the last minute of the game and the team that is currently loosing has a corner. They therefore decide to take out the goalie and use him as an attacker as well. The corner isn't successful, the other team counterattacks and shoots from 50 meters into the empty goal. Did this shot happen in reality? Yes, of course, it did! But do we want the model to learn from that shot? I guess it depends. If we had tracking data and knew that the goal was empty, maybe yes. If we don't have this data, however, then most arguably not. We do not want the model to learn that there is a chance of scoring when shooting the ball from 50 meters. And unfortunately, taking care of this is even more important as we have the bias that we had talked about above, namely that a player only shoots from 50 meters if there is no goalie - and therefore has a realistic chance of making it. 

Ok, enough of the talking, let's go through the different variables

#### Position of the shot

We start by identifying outliers regarding the position of the shots. Let's therefore first plot a heatmap of where all the shots happened.

In [74]:
# compute the number of shots per grid
nb_shots, x, y, df_shots = py_help.prepare_heatmap(df_shots, "posBeforeXMeters", "posBeforeYMeters", 24, 17, return_df=True)
# get the share of shots
share_shots = nb_shots / nb_shots.sum() * 100
# plot the heatmap
dict_info = {"Share of shots (in %)": {"values": share_shots, "display_type": ".2f"},
             "Number of shots": {"values": nb_shots, "display_type": ".0f"}}
fig = py_help.create_heatmap(x, y, share_shots, dict_info, title_name="Position of shots taken")
fig.show()

Ok, that doesn't look too bad. Most of the shots where taken either inside or slightly outside the box. That makes a lot of sense. When hovering over the different cells, however, you might already notice that there are some shots taken from the own half... Let's make this clearer when looking at the probability to score for the different cells.

In [75]:
# get all shots that ended in a goal
df_goals = df_shots[df_shots["goal"] == 1].copy()

# compute the number of shots per grid
nb_goals, x, y = py_help.prepare_heatmap(df_goals, "posBeforeXMeters", "posBeforeYMeters", 24, 17)

# compute the share of goals per grid cell
goal_proba = np.divide(nb_goals, nb_shots, out=np.zeros_like(nb_goals), where=nb_shots!=0) * 100

# plot the heatmap
dict_info = {"Scoring probability (in %)": {"values": goal_proba, "display_type": ".1f"},
             "Share of shots (in %)": {"values": share_shots, "display_type": ".2f"},
             "Number of shots": {"values": nb_shots, "display_type": ".0f"},
             "Number of goals": {"values": nb_goals, "display_type": ".0f"}}
fig = py_help.create_heatmap(x, y, goal_proba, dict_info, title_name="Probability to score")
fig.show()

Whut?! What the hell is this? The zones with the highest likelihood to score are from the own goal area, the right midfield as well as close to the corner flag?! That is definitely wrong! 

You see what the problem is? While the data points in the own goal area are almost certainly wrong, the other might indeed have happened in a match and are therefore right. However, when e.g. looking at the cell in the right midfield: There was one shot and that shot went in. Assuming this situation indeed happened, I'm pretty sure that either there was no goalie or the goalie was extremely badly positioned (unfortunately we cannot tell from the data we have). The attacker saw this and therefore decided to shoot but would have never shot otherwise. So do we really want to have a model that tells us that there is a 100% chance of making it when shooting from the right midfield (because this is what a perfect model trained on this data would do)? I guess we don't, do we? 

So what can we do with those data points? 

1. We could just delete them
2. We could use our experience and tell the model that the probability to score from this position is realistically 0. This means we change the target but leave the features as is.
3. We assume that the shot happened closer to the goal, but we still assume that it went in. This means that we change the features but leave the target.

While I would love to give you one generic answer to this questions, unfortunately there isn't any. The strategy that is generally most often used is 3, which is called [winsorization](https://en.wikipedia.org/wiki/Winsorizing). In our situation, however, I would argue that the second strategy makes the most sense. So let's do this...

We first quickly create a heatmap that gives us the chance of shooting vs. passing for the different cells. That way we can get a feeling in which zones players rather decide to pass than to shoot and vice versa

In [76]:
df_passes = df_events[df_events["eventName"] == "Pass"].copy()

# compute the number of passes per grid
nb_passes, x, y, df_passes = py_help.prepare_heatmap(df_passes, "posBeforeXMeters", "posBeforeYMeters", 24, 17, 
                                                     return_df=True)

nb_passes_and_shots = nb_passes + nb_shots

# compute the share of shots per grid cell
shot_proba = np.divide(nb_shots, nb_passes_and_shots, out=np.zeros_like(nb_shots), where=nb_passes_and_shots!=0) * 100

# plot the heatmap
dict_info = {"Probability to shoot (in %)": {"values": shot_proba, "display_type": ".2f"},
             "Number of shots": {"values": nb_shots, "display_type": ".0f"},
             "Number of passes": {"values": nb_passes, "display_type": ".0f"}}
fig = py_help.create_heatmap(x, y, shot_proba, dict_info, title_name="Shooting probability (in %)")
fig.show()

That heatmap is completely aligned with our intuition, isn't it? 

I would now claim the following: If for a cell out of 100 situations where a player can either shoot or pass, on average he decides to pass more than 99 times, this is because the chance of making a goal when shooting is (close to) 0. Given that, we are therefore going to assume that all shots happening in any of the cells with < 1% shooting probability did not result in a goal.

In [77]:
group_cols = ["posBeforeXMetersZone","posBeforeYMetersZone"]

# get the number of shots per grid cell
df_shots_per_cell = df_shots.groupby(group_cols).\
                             agg(nbShots=("id","count")).reset_index()

# get the number of passes per grid cell
df_passes_per_cell = df_passes.groupby(group_cols).\
                               agg(nbPasses=("id","count")).reset_index()

# merge the two and compute the probability to shoot (compared to passing) for each cell
df_shots_passes = pd.merge(df_shots_per_cell, df_passes_per_cell, how="outer")
df_shots_passes["nbShots"].fillna(0, inplace=True)
df_shots_passes["nbPasses"].fillna(0, inplace=True)
df_shots_passes["shotProbability"] = df_shots_passes["nbShots"] / (df_shots_passes["nbShots"] + df_shots_passes["nbPasses"])

# merge to all the shots
df_shots = pd.merge(df_shots, df_shots_passes[group_cols + ["shotProbability"]], how="left")

# save the number of goals before the change
nb_goals_before = sum(df_shots["goal"])

# whenever the shot probability for this cell is < 1%, we assume that the shot did not result in a goal
df_shots["goal"] = np.where(df_shots["shotProbability"] < 0.01, 0, df_shots["goal"])

change_goals = nb_goals_before - df_shots['goal'].sum()
print(f"Relabeled {change_goals} goals. This is{change_goals / nb_goals_before * 100: .2f}% of the goals and{change_goals / len(df_shots) * 100: .2f}% of the shots.")

Relabeled 26 goals. This is 0.61% of the goals and 0.06% of the shots.


Ok, we relabeled 26 goals which is ~ every 160th goal. I guess that is something that you can indeed call an outlier (and don't forget, we only do it for the good of the model). Let's see how the probability to score looks now.

In [78]:
# compute the number of shots per grid
nb_shots, x, y, df_shots = py_help.prepare_heatmap(df_shots, "posBeforeXMeters", "posBeforeYMeters", 24, 17, return_df=True)
# get the share of shots
share_shots = nb_shots / nb_shots.sum() * 100

# get all the goals
df_goals = df_shots[df_shots["goal"] == 1].copy()

# compute the number of shots per grid
nb_goals, x, y = py_help.prepare_heatmap(df_goals, "posBeforeXMeters", "posBeforeYMeters", 24, 17)

# compute the share of goals per grid cell
goal_proba = np.divide(nb_goals, nb_shots, out=np.zeros_like(nb_goals), where=nb_shots!=0) * 100

# plot the heatmap
dict_info = {"Scoring probability (in %)": {"values": goal_proba, "display_type": ".1f"},
             "Share of shots (in %)": {"values": share_shots, "display_type": ".2f"},
             "Number of shots": {"values": nb_shots, "display_type": ".0f"},
             "Number of goals": {"values": nb_goals, "display_type": ".0f"}}
fig = py_help.create_heatmap(x, y, goal_proba, dict_info, 
                             title_name="Probability to score", 
                             colour_scale=(0,40))
fig.show()

This looks pretty reasonable now, doesn't it? Nice, so we already took a look at the positional outliers. So what about the other features?

#### Body part

Let's quickly have a look at the values and the count for the different body parts

In [79]:
df_shots.groupby("bodyPartShot").size()

bodyPartShot
head/body     6519
leftFoot     13323
rightFoot    20619
dtype: int64

Hm, nothing weird here, is there? So we can just leave everything as is...

#### Counter attack

In [80]:
df_shots.groupby("counterAttack").size()

counterAttack
0    38210
1     2251
dtype: int64

Nice, same story as for the body part. Again nothing to do :-)

#### Impact on the log loss

Before we start with the next section, let's quickly check what impact the outlier treatment had on our log loss. As we only changed 0.06% of the data points, we would only expect a very little impact. Nevertheless, we should recompute it to have a good baseline for our actual model development.

First, we need to recalculate our training and test set (by the way, if you don't know why we split into test and train, you should definitely look it up - this is one of the key concepts of machine learning and there is tons of material on this online). Notice, that in contrast to the shitty example above, we do not only keep the feature columns and target column separately, but we store everything (including non-feature information) in a new data frame. We will see below why in my opinion this makes a lot of sense...

In [81]:
df_train, df_test, _, _ = train_test_split(df_shots, df_shots["goal"], test_size=0.25, random_state=42)
df_train.head()

,id,matchId,matchPeriod,eventSec,eventName,subEventName,teamId,posBeforeXMeters,posBeforeYMeters,posAfterXMeters,posAfterYMeters,playerId,playerName,playerPosition,playerStrongFoot,teamPossession,homeTeamId,awayTeamId,goal,ownGoal,counterAttack,bodyPartShot,bodyPartShotCode,posBeforeXMetersZone,posBeforeYMetersZone,shotProbability
20686,220884257,2499939,1H,1927.767965,Shot,Shot,1609,76.65,32.64,105.0,68.0,49876,G. Xhaka,MD,left,1609,1659,1609,0,0,0,leftFoot,1,17,8,0.148962
30455,241111849,2565852,1H,776.495085,Shot,Shot,675,95.55,31.28,0.0,0.0,8278,G. Bale,FW,left,675,675,679,0,0,0,head/body,2,21,7,0.856164
23556,247255947,2500071,1H,2057.544092,Shot,Shot,1673,92.40,29.92,0.0,0.0,214654,S. Mounié,FW,right,1673,1673,1623,0,0,0,head/body,2,21,7,0.856164
2764,211492829,2516858,1H,1358.771285,Shot,Shot,2462,91.35,22.44,105.0,68.0,69616,A. Rebić,FW,right,2462,2457,2462,0,0,0,rightFoot,3,20,5,0.503254
29539,231574469,2565806,2H,388.734609,Shot,Shot,680,96.60,38.08,105.0,68.0,70125,Nolito,FW,right,680,683,680,0,0,0,rightFoot,3,22,9,0.933390


Let's save the raw training and test data sets so that we can use them later on

In [82]:
df_train_raw = df_train.copy()
df_test_raw = df_test.copy()

We re-train the logistic regression and check the log loss on the test set

In [83]:
features = ["posBeforeXMeters", "posBeforeYMeters", "counterAttack", "bodyPartShotCode"]

# training of the logistic regression on the train set
reg_outlier = LogisticRegression(random_state=42)
reg_outlier.fit(df_train[features], np.array(df_train["goal"]).ravel())

# prediction on the test set
pred_probs = reg_outlier.predict_proba(df_test[features])[:,1]
print(f"Log loss after outlier removal: {sk_metrics.log_loss(df_test['goal'], pred_probs):.5f}")

Log loss after outlier removal: 0.30299


Good, the log loss indeed only changed very slightly from 0.30533 to 0.30299, i.e. it was reduced by 0.8%. Given that we only changed 0.06% of the data points, this seems reasonable! 

Great, we are now ready to final start the actual work, namely the building of our expected goal model! :-)

<a id="univariate"></a>

# 4. Univariate analysis

Finally, we are going to start with building our model! :-) 

In the univariate analysis part we are going to go through the variables we have already worked with and focus on the following things:
1. Meaningfulness of features
2. Monotonicity of continuous features
3. Dummy encoding of categorical features

This might sound like very few steps but I promise that be the end of this section we will have already improved the log loss by quite a bit :-)

#### X Position of the shot

Let's start by looking at the x-coordinate of the position where the shot was taken, i.e. *posBeforeXMeters*. First of all, we should ask ourselves if the variable is indeed meaningful. Using meaningful features makes it 

- Way easier for you to sense-check results
- Quite often helps you to come up with better features
- Helps to explain the model to others

While the distance from the own goal is somehow meaningful, I believe switching it to the distance to the opponents goal line is way more meaningful, isn't it? So let's do this (notice that we will draw all conclusions below only on *df_train* and leave *df_test* untouched)

In [84]:
df_train = df_train.copy()
df_train["distToGoalLine"] = 105 - df_train["posBeforeXMeters"]

When using a logistic regression, it is crucial that the probabilities are montone in the continuous variables. So let's quickly check if this is the case for our features (if you did not get the last sentence, don't worry, I'll explain it again below with the help of a picture). We can do this by using two helper functions. 

In [85]:
df_train["distToGoalLineBuckets"] = ml_help.build_buckets(df_train, "distToGoalLine", 
                                                  step_size=2, 
                                                  min_val=1,
                                                  max_val=34)

fig, df_group = ml_help.create_univariate_variable_graph(df=df_train, 
                                               col="distToGoalLineBuckets", 
                                               target_col="goal", 
                                               binned_cols=True,
                                               title_name="Distance to the goal line in meter")
fig.show()

Nice, that looks very good! We see that the further the shot is from the goal line, the less likely it is to score. That makes a lot of sense and is exactly what we would expect! :-) 

So what about the (strong) monotonicity, why do we need this? The "problem" with logistic regression is that it can only make statements like
- The further from the goal line the worse
- The sooner after the opponent lost the ball the better
- The older the player the better 

, but no statements like
- When the distance to the goal line is <= 10 meters it is good, between 10 - 16 meters it gets worse but then when more than 16 meters it gets better again
- The first 10 seconds after the opponent lost the ball is very good, but then it does not matter any more
- Things get better until the player turns 28 and then start to decrease

You see the difference in the statements? Fortunately, the distance to the goal line is strong monotone as we see above. So no need to change anything for now. And don't worry, even if we have features that are like the ones below, there are ways to convert them into monotone features. As we see in the next example. :-)

#### Y position of the shot

Let's start by plotting the univariate variable graph of the y-coordinate of the shots

In [86]:
df_train["posBeforeYMetersBuckets"] = ml_help.build_buckets(df_train, "posBeforeYMeters", 
                                                    step_size=3, 
                                                    min_val=10, 
                                                    max_val=58)

fig, df_group = ml_help.create_univariate_variable_graph(df=df_train, 
                                                         col="posBeforeYMetersBuckets", 
                                                         target_col="goal", 
                                                         binned_cols=True,
                                                         title_name="Y-Coordinate on the field")
fig.show()

Hm, that does not look monotone, does it? But how could we change it to a monotone feature? Let's see what it tells us: Until meter ~34 the probability to score increases and after it is decreases. Given that meter 34 is the center of the field (remember that the field has a width of 68 meters) one could also say: The further away from the center the less likely to score. This statements is 
- monotone :-)
- way easier to interpret than the other one

So, let's build the new feature

In [87]:
df_train["distToCenter"] = np.abs(34 - df_train["posBeforeYMeters"])

In [88]:
df_train["distToCenterBuckets"] = ml_help.build_buckets(df_train, "distToCenter", 
                                                  step_size=2,
                                                  max_val=20)

fig, df_group = ml_help.create_univariate_variable_graph(df=df_train, 
                                               col="distToCenterBuckets", 
                                               target_col="goal", 
                                               binned_cols=True,
                                               title_name="Distance to center of the field")
fig.show()

Nice, this looks way better, doesn't it? 

I have to admit that I made this chart look slightly better than it looks when zooming in. You noticed that I set *step_size* to 2 and *max_val* to 20 in the *build_buckets* function above? Let's look at what happens if we use *step_size* of 1 and no *max_val*

In [89]:
df_train["distToCenterBuckets"] = ml_help.build_buckets(df_train, "distToCenter", 
                                                  step_size=1)

fig, df_group = ml_help.create_univariate_variable_graph(df=df_train, 
                                               col="distToCenterBuckets", 
                                               target_col="goal", 
                                               binned_cols=True,
                                               title_name="Distance to center of the field")
fig.show()

Hm, that does not look that great any more, does it? Two things we immediately see

1. The blue bars for the share of observations look kind of weird with e.g. 2-3 being big, 3-4 small and then 4-5 big again
2. The orange line for the target probability is not monotone any more

For the first point, the reason is simple. It is just the accuracy of measurement in the underlying data. As the position is not automatically retrieved, but most likely hand-labeled, positions are rather (non-accurate) estimates than very true positions. What about the second point, however? Truth is, that in reality, when you zoom deep enough, you will almost always find some non-monotonicity in the data. It is your job as a data scientist or soccer analyst to decide whether this is still acceptable or whether you need to do something about it. This will come with experience, but as a rule of thumb, I would say that if you can find a monotone graph with a reasonable share for each bucket (i.e. the blue bars) things are fine. Also, you should most probably only do something about it, if the non-monotonicity makes sense from an expert judgement! What I mean is the following: Mathematically we could transform the above in such a way that the probability of scoring decreases up to 19 meters, then increases from 19 - 22 meters and then falls to 0. But is it really realistic, that it is more likely to score if you are 21 meters from the center compared to 19 meters? I would highly doubt that...  

#### Body part

The next variable we want to look at is the body part with which the shot was taken. Let's start by looking at whether this variable is meaningful and quickly reminding ourselves on the different values this variable takes.

In [90]:
df_train.groupby("bodyPartShot").size()

bodyPartShot
head/body     4869
leftFoot     10011
rightFoot    15465
dtype: int64

Ok, there is left foot, right foot and header. So, imagine yourself explaining your model to the coach and telling him that e.g. a shot had a higher chance of going in, because it was taken with the right foot. Does this sound reasonable? I guess in most cases it does. However, imagine you were talking to Roberto Carlos' coach and told him that. He would most likely tell you that this is complete non-sense as Roberto's strong foot is the left foot. So instead of looking at whether the shoot was taken with the left or right foot, wouldn't it make more sense to look at whether or not the shot was taken with the strong foot?!

In [91]:
# check for the different values of *playerStrongFoot*
df_train.groupby("playerStrongFoot").size()

playerStrongFoot
both          71
left        7452
right      22812
unknown        7
dtype: int64

Let's now transform the body part to *strongFoot*, *weakFoot* and *head/body*

In [92]:
def transform_body_part_shot(row):
    """
    Helper function to identify whether a shot was taken with the strong or the weak foot.
    """
    if row["bodyPartShot"] == "head/body":
        return "head/body"
    elif row["bodyPartShot"] == "rightFoot":
        if row["playerStrongFoot"] == "left":
            return "weakFoot"
        else: 
            return "strongFoot"
    elif row["bodyPartShot"] == "leftFoot":
        if row["playerStrongFoot"] == "right":
            return "weakFoot"
        else: 
            return "strongFoot"
    else:
        raise ValueError(f"Body part *{row['bodyPartShot']}* unknown.")

In [93]:
df_train["bodyPartShot"] = df_train.apply(transform_body_part_shot, axis=1)

Nice, now we got a way more meaningful feature, didn't we? 

Notice, however, that the body part is a [categorical variable](https://stats.idre.ucla.edu/other/mult-pkg/whatstat/what-is-the-difference-between-categorical-ordinal-and-numerical-variables/), i.e. it is a variable where the different values do not have any meaningful order. As logistic regression, however, always expects an ordering in the feature, <b> we always need to encode categorical variables </b> when using logistic regression. There are different strategies to encode, but unless you are more experienced and know what you are doing, I would just use [dummy encoding](https://www.statisticssolutions.com/dummy-coding-the-how-and-why/) (also called one-hot encoding).

In [94]:
# get the dummy variables
df_dummy_body_part = pd.get_dummies(df_train["bodyPartShot"])
# attach them to the training dataset
df_train = pd.concat([df_train, df_dummy_body_part], axis=1)

Cool, done with the dummy encoding, so let's go to the next variable

#### Counter attack

Fortunately, for the counter attack there is nothing to do, as the counter attack is a binary variable which are usually most easy to handle :-) Nevertheless, let's quickly have a look if the chance of scoring is higher after a counter attack.

In [95]:
fig, _ = ml_help.create_univariate_variable_graph(df=df_train, 
                                               col="counterAttack", 
                                               target_col="goal", 
                                               binned_cols=False,
                                               title_name="Chance of scoring after counter attack")
fig.show()

It indeed is! While there is a 10.2% chance of a ball going in, when there is no counter attack, this raises to 14.2% in case of a counter attack.

Now, finally, let's jump into the model performance and check whether all of the work we did above paid off...

#### Model performance

Notice that so far we have only transformed the training dataset but not the test dataset. So let's start by quickly writing a small function that does all of the transformation steps defined above.

In [96]:
def transformation_univariate(df):
    """
    Function performs all transformation steps defined in the section *Univariate analysis - Part A*
    """
    
    df = df.copy()
    
    # compute the distance to the goal line
    df["distToGoalLine"] = 105 - df["posBeforeXMeters"]
    
    # compute the distance to the center
    df["distToCenter"] = np.abs(34 - df["posBeforeYMeters"])
    
    # convert body part to strong foot / weak foot
    df["bodyPartShot"] = df.apply(transform_body_part_shot, axis=1)
    
    # dummy encode the body part
    df_dummy_body_part = pd.get_dummies(df["bodyPartShot"])
    # attach them to the training dataset
    df = pd.concat([df, df_dummy_body_part], axis=1)
    
    return df

Transform the raw training and test datasets

In [97]:
df_train_trans = transformation_univariate(df_train_raw)
df_test_trans = transformation_univariate(df_test_raw)

Let's quickly check whether we implemented our transformation function correctly by making sure, that the values between the "manually" changed *df_train* and *df_train_trans* match. 

In [99]:
features = ["distToGoalLine", "distToCenter", "head/body", "strongFoot", "weakFoot", "counterAttack"]
print(f"Transformation done correctly: {ml_help.check_column_match(df_train, df_train_trans, features)}")

Transformation done correctly: True


Nice, so let's re-train the model and see how the log loss on the test set evolved

In [100]:
# training of the logistic regression on the train set
reg_uni_a = LogisticRegression(random_state=42)
reg_uni_a.fit(df_train_trans[features], np.array(df_train_trans["goal"]).ravel())

# prediction on the test set
pred_probs = reg_uni_a.predict_proba(df_test_trans[features])[:,1]
print(f"Log loss on test set after univariate analysis part A: {sk_metrics.log_loss(df_test_trans['goal'], pred_probs):.5f}")

Log loss on test set after univariate analysis part A: 0.28504


Great! We were able to reduce the log loss from 0.303 to 0.285; this is an increase by ~6%. Not bad...

For comparison, let's also compute the error on the train set and see if we also increased the AUC on the test set

In [101]:
pred_probs_train = reg_uni_a.predict_proba(df_train_trans[features])[:,1]
df_train_trans["prediction"] = pred_probs_train
print(f"Log loss on train set after univariate analysis part A: {sk_metrics.log_loss(df_train_trans['goal'], pred_probs_train):.5f}")
print(f"AUC on test set after univariate analysis part A: {sk_metrics.roc_auc_score(df_test_trans['goal'], pred_probs)*100:.2f}%")

Log loss on train set after univariate analysis part A: 0.28453
AUC on test set after univariate analysis part A: 78.01%


Cool, even though - as we said above - the AUC is not the ultimate measure, it is still cool to see the value increased significantly from 73.3% to 78.0%.

<a id="features"></a>

# 5. Creating new features

Arguably, the most important task that you have to do when building models, is the come up with good features. In this section we are therefore going to build new features and try to reduce the log loss even more.

Let's start by computing some obvious features, like having a corner kick beforehand.

In [102]:
df_train = df_train_trans.copy()
df_test = df_test_trans.copy()

#### Corner kick

Let's build a feature that tells us whether the shot happened after a corner kick. Before we do that, however, we first need to extract the event before the shot

In [103]:
def compute_event_before(df_events):
    df = df_events.copy()
    
    df["eventBefore"] = df.groupby(["matchId", "matchPeriod"])["eventName"].shift(1)
    df["subEventBefore"] = df.groupby(["matchId", "matchPeriod"])["subEventName"].shift(1)
    df["teamBefore"] = df.groupby(["matchId", "matchPeriod"])["teamId"].shift(1)
    
    return df

def add_feature_corner(df, df_events):
    
    if "corner" in df.columns:
        df.drop("corner", axis=1, inplace=True)
     
    df_corner = df_events[df_events["subEventBefore"] == "Corner"].copy()
    df_corner["corner"] = 1
    df = pd.merge(df, df_corner[["id", "corner"]], on="id", how="left")
    df["corner"].fillna(0, inplace=True)
    return df 

# get a data frame with the events before each event
df_before = compute_event_before(df_events)

# add a feature whether the shot was directly after a corner
df_train = add_feature_corner(df_train, df_before)
df_test = add_feature_corner(df_test, df_before)

In [104]:
fig, _ = ml_help.create_univariate_variable_graph(df=df_train, 
                                                  col="corner", 
                                                  target_col="goal", 
                                                  binned_cols=False,
                                                  title_name="Chance of scoring after corner kick")
fig.show()

Ok, that makes a lot of sense. The chance of scoring is lower after a corner kick as the box is pretty crowded. 

#### Cross

Add a feature that tells us whether the ball came from a cross

In [105]:
def add_feature_cross(df, df_events):
    
    if "cross" in df.columns:
        df.drop("corner", axis=1, inplace=True)
    
    df_cross = df_events[df_events["subEventBefore"] == "Cross"].copy()
    df_cross["cross"] = 1
    df = pd.merge(df, df_cross[["id", "cross"]], on="id", how="left")
    df["cross"].fillna(0, inplace=True)
    return df    

# add a feature whether the shot happened after a cross
df_train = add_feature_cross(df_train, df_before)
df_test = add_feature_cross(df_test, df_before)

In [106]:
fig, _ = ml_help.create_univariate_variable_graph(df=df_train, 
                                                  col="cross", 
                                                  target_col="goal", 
                                                  binned_cols=False,
                                                  title_name="Chance of scoring after a cross")
fig.show()

Hm, ok, there seems to be a way higher chance to score after a cross. This seems somehow odd, but I guess if the data tells us so... 

#### Error reduction

Wait, let's stop here for a minute. Prize question: What do you think, which variable will have a bigger relative impact on our model, the corner variable, which tells us that the probability of scoring after a corner decreases from 10.5% to 8%, or the cross variable, that tells us that the probability of scoring raises to 17.9% after a cross? 

Let's investigate for a little bit. So where do we stand in the process of building our model and what is our purpose of searching for new features? We already have a pretty decent model and try to improve it by finding *additional* features that are capabable of reducing our error. So is the variable cross capable to do so? We don't know yet, but we can easily find out. 

We first compute the mean chance of scoring a goal after a cross (this is the same information as in the chart above)

In [107]:
df_train.groupby("cross")["goal"].mean()

cross
0.0    0.096685
1.0    0.179084
Name: goal, dtype: float64

Now, let's compute the mean prediction after a cross

In [108]:
df_train.groupby("cross")["prediction"].mean()

cross
0.0    0.095874
1.0    0.186664
Name: prediction, dtype: float64

You see what I'm seeing? For the cross balls we are already predicting that there is a 18.7% chance of scoring. This is actually even higher than what we see in reality (17.9%), i.e. a cross has most likely a negative impact rather than a positive one! Actually it seems like our model is already doing a pretty good job and adding the cross variable will not have a huge impact (or at least not as much as we might have though after looking at the univariate chart above). 

Let's visualize this by adjusting the target in our univariate graph

In [109]:
df_train["overprediction"] = df_train["prediction"] - df_train["goal"]
fig_cross, _ = ml_help.create_univariate_variable_graph(df=df_train, 
                                                  col="cross", 
                                                  target_col="overprediction", 
                                                  binned_cols=False,
                                                  y2_axis_name="Overprediction (in pp)",
                                                  title_name="Overprediction for crosses")
fig_cross.show()

Let me quickly explain the orange line above: This lines shows us the overprediction of our model. This means that after a cross the model predicts on average a 0.76 pp higher probability of scoring than we observe it in reality. Makes sense?

Ok, let's do the same exercise for the corners

In [110]:
df_train.groupby("corner")["goal"].mean()

corner
0.0    0.105237
1.0    0.079890
Name: goal, dtype: float64

In [111]:
df_train.groupby("corner")["prediction"].mean()

corner
0.0    0.104162
1.0    0.123674
Name: prediction, dtype: float64

In [112]:
fig_corner, _ = ml_help.create_univariate_variable_graph(df=df_train, 
                                                  col="corner", 
                                                  target_col="overprediction", 
                                                  binned_cols=False,
                                                  y2_axis_name="Overprediction (in pp)",
                                                  title_name="Overprediction for corners")
fig_corner.show()

For comparison, let's draw the two into one chart with the same y-axis

In [113]:
ml_help.combine_univariate_variable_graphs([fig_cross, fig_corner], 2, 1, shared_axis=True)

The answer to the question above is not that obvious any more, is it? ;-) Let's quickly check how the log loss looks like after adding the two features

In [114]:
# running the logistic regression when adding variables for cross or corner
for variable in ["cross", "corner"]:

    features = ["distToGoalLine", "distToCenter", "head/body", "strongFoot", "weakFoot", "counterAttack"] + [variable]

    # training of the logistic regression on the train set
    reg = LogisticRegression(random_state=42)
    reg.fit(df_train[features], np.array(df_train["goal"]).ravel())

    # prediction on the test set
    pred_probs = reg.predict_proba(df_test[features])[:,1]
    print(f"Log loss on test set when adding {variable}: {sk_metrics.log_loss(df_test['goal'], pred_probs):.5f}")

Log loss on test set when adding cross: 0.28508
Log loss on test set when adding corner: 0.28437


Interesting, indeed, we are only capable of reducing the error on the test set when adding a variable for corner! Adding the cross variable does not make any difference in the log loss...

So what's the conclusion out of all of the above: 

<b> When searching for additional features, prioritize features that can reduce the error your current model makes over features, that explain the target well </b>

Having said this, you should obviously still always plot the target plot. Not necessarily because it will tell you an awful lot on whether or not a feature is useful, but it can help you a lot in validating the feature and making sure that there are no bugs in the feature creation (trust me, those happen more often than you might think)

Ok, some of you might now say, that the above is obvious as there is a huge correlation between the *cross* variable and the other features. While I completely agree, that there is some relationship (if the variables were completely independent, one wouldn't observe what we saw above), it is in reality rather difficult to see it with the usually used methods. To show you what I mean, let's quickly compute the correlations as well as the VIF for the two variables (if you don't know what those are, yet, don't worry, we will talk more about those later on)

In [115]:
# quickly check for the correlation between the features
for variable in ["cross", "corner"]:
    print(f"Correlations of {variable} with the other features\n####################")
    for feat in features:
        if variable == feat:
            continue
        print(f"{feat}: {np.corrcoef(df_train[feat], df_train[variable])[0][1] * 100: .1f}%")
    print("\n")

features = ["distToGoalLine", "distToCenter", "head/body", "strongFoot", "weakFoot", "counterAttack", "cross", "corner"] 
df_feat = df_train[features].copy()

print("Variance inflation factors\n####################")
for i in [6,7]:
    vif = variance_inflation_factor(df_feat.values, i)
    print(f"VIF for {features[i]}: {vif:.2f}")

Correlations of cross with the other features
####################
distToGoalLine: -23.8%
distToCenter: -17.0%
head/body:  18.9%
strongFoot: -13.3%
weakFoot: -1.5%
counterAttack: -2.2%
corner: -5.1%


Correlations of corner with the other features
####################
distToGoalLine: -8.7%
distToCenter: -3.7%
head/body:  17.1%
strongFoot: -8.9%
weakFoot: -5.1%
counterAttack: -3.7%


Variance inflation factors
####################
VIF for cross: 1.10
VIF for corner: 1.04


While there is some correlation to the other features, it isn't huge, is it?

Ok, enough with this topic for now. Let's instead continue to build a couple of features.

#### Smart pass

In [116]:
def add_feature_smart_pass(df, df_events):
    
    if "smartPass" in df.columns:
        df.drop("smartPass", axis=1, inplace=True)
    
    df_smart_pass = df_events[df_events["subEventBefore"] == "Smart pass"].copy()
    df_smart_pass["smartPass"] = 1
    df = pd.merge(df, df_smart_pass[["id", "smartPass"]], on="id", how="left")
    df["smartPass"].fillna(0, inplace=True)
    return df  

# add a feature whether the shot happened after a smart pass
df_train = add_feature_smart_pass(df_train, df_before)
df_test = add_feature_smart_pass(df_test, df_before)

# probability graph after smart pass
fig_prob, _ = ml_help.create_univariate_variable_graph(df=df_train, 
                                                  col="smartPass", 
                                                  target_col="goal", 
                                                  binned_cols=False,
                                                  title_name="Chance of scoring after smart pass")


fig_pred, _ = ml_help.create_univariate_variable_graph(df=df_train, 
                                                  col="smartPass", 
                                                  target_col="overprediction", 
                                                  binned_cols=False,
                                                  y2_axis_name="Overprediction (in pp)",
                                                  title_name="Overprediction for smart passes")

ml_help.combine_univariate_variable_graphs([fig_prob, fig_pred], 2, 1, shared_axis=False)

Hm, it looks like there is a significant underprediction after smart passes, i.e. the model significantly underestimates the chance of scoring after a smart pass. Definitely looks like a variable we should keep...

#### Duels

Let's check whether there is any purpose in keeping a feature that tells us whether or not there was a duel right before the shot.

In [117]:
def add_feature_duel(df, df_events):
    
    if "duel" in df.columns:
        df.drop("duel", axis=1, inplace=True)
    
    df_duel = df_events[df_events["eventBefore"] == "Duel"].copy()
    df_duel["duel"] = 1
    df = pd.merge(df, df_duel[["id", "duel"]], on="id", how="left")
    df["duel"].fillna(0, inplace=True)
    return df  

# add a feature whether the shot happened after a smart pass
df_train = add_feature_duel(df_train, df_before)
df_test = add_feature_duel(df_test, df_before)

# probability graph after smart pass
fig_prob, _ = ml_help.create_univariate_variable_graph(df=df_train, 
                                                  col="duel", 
                                                  target_col="goal", 
                                                  binned_cols=False,
                                                  title_name="Chance of scoring after duel")


fig_pred, _ = ml_help.create_univariate_variable_graph(df=df_train, 
                                                  col="duel", 
                                                  target_col="overprediction", 
                                                  binned_cols=False,
                                                  y2_axis_name="Overprediction (in pp)",
                                                  title_name="Overprediction for duels")

ml_help.combine_univariate_variable_graphs([fig_prob, fig_pred], 2, 1, shared_axis=False)

So, what do we do with this feature? On the first sight, it might look as if it is basically the same story as with the crosses.  Overprediction of crosses was at ~0.75pp and overprediction for duels is at ~0.9pp. However, there is one very interesting difference: Look at the underprediction of non-crosses and non-duels, respectively. While there was basically no underprediction for non-crosses, the underprediction for non-duels is at ~ 0.6pp. This means that the difference in error between non-duel and duel is ~1.5pp while it was only ~0.8pp between non-crosses and crosses (all of this is obviously due to the fact that there are way more duels than crosses). So it probably makes sense to keep this feature as well, doesn't it? 

#### Shot before

For now, let's add one additional feature telling us, whether there was a shot or goalie reflex right before the shot we are looking at.

In [118]:
def add_feature_shot_before(df, df_events):

    if "shotBefore" in df.columns:
        df.drop("shotBefore", axis=1, inplace=True)
    
    df_shot_before = df_events[df_events["subEventBefore"].isin(["Shot", "Reflexes", "Save attempt"])].copy()
    df_shot_before["shotBefore"] = 1
    df = pd.merge(df, df_shot_before[["id", "shotBefore"]], on="id", how="left")
    df["shotBefore"].fillna(0, inplace=True)
    return df   

# add a feature whether the shot happened after a smart pass
df_train = add_feature_shot_before(df_train, df_before)
df_test = add_feature_shot_before(df_test, df_before)

# probability graph after smart pass
fig_prob, _ = ml_help.create_univariate_variable_graph(df=df_train, 
                                                  col="shotBefore", 
                                                  target_col="goal", 
                                                  binned_cols=False,
                                                  title_name="Chance of scoring after another shot")


fig_pred, _ = ml_help.create_univariate_variable_graph(df=df_train, 
                                                  col="shotBefore", 
                                                  target_col="overprediction", 
                                                  binned_cols=False,
                                                  y2_axis_name="Overprediction (in pp)",
                                                  title_name="Overprediction for after other shots")

ml_help.combine_univariate_variable_graphs([fig_prob, fig_pred], 2, 1, shared_axis=False)

Ok, I guess we again should keep this feature for now. However, when looking at the number of observations, this is becoming somehow shady as we haven't observed this a lot at all. Let's keep this feature for now but keep in mind that good feature are usually such, that also occur regularly. 

Nice, now we have built a couple of new features! Before we continue to build one more feature, let's quickly check if we see any improvement from the new features.

#### Model performance after event features

Notice that we do not consider the cross feature - if you want to add it, please go ahead. You will see that it does indeed not improve the performance.

In [119]:
features = ["distToGoalLine", "distToCenter", "head/body", "strongFoot", "weakFoot", "counterAttack", 
            "corner", "smartPass", "duel", "shotBefore"]

# training of the logistic regression on the train set
reg_feat_1 = LogisticRegression(random_state=42)
reg_feat_1.fit(df_train[features], np.array(df_train["goal"]).ravel())

# prediction on the test set
pred_probs = reg_feat_1.predict_proba(df_test[features])[:,1]
print(f"Log loss on test set after event features: {sk_metrics.log_loss(df_test['goal'], pred_probs):.5f}")

Log loss on test set after event features: 0.28272


Nice, again we managed to improve the model! I hear you, you are saying now: "Wtf, all of this work to increase from 0.28453 to 0.28272?" But to be honest, improving models that are already somehow good by a huge margin is rather difficult (and our model is already good). And, we have not really spend a lot of time trying to really optimize the features, have we? Let's take the corners for example. What about all the situations where there was a corner and one attacker had passed the ball to another one. We did not try to also label those as corners, did we?

Before we continue with the next steps, let's make sure we update our model predictions

In [120]:
df_train["prediction"] = reg_feat_1.predict_proba(df_train[features])[:,1]
df_train["overprediction"] = df_train["prediction"] - df_train["goal"]

#### Angle of the shot

So what we did above, was, to first come up with an idea for a feature and then check whether we think that it can improve the performance of our model by looking at the model errors. Often, however, it can also be very useful to start the other way round, i.e. look at the errors to get ideas on potential additional features. 

Let's therefore create a heatmap to see where we positionally currently make the biggest error. We do so by computing the log loss for each zone of the field

In [121]:
def log_loss(df, target_col, pred_col, eps=1e-15):
    df = df.copy()
    df["predClip"] = df[pred_col].clip(lower=eps, upper=1-eps)
    df["logLoss"] = np.where(df[target_col] == 1, -1*np.log(df["predClip"]), -1*np.log(1-df["predClip"]))
    df.drop("predClip", axis=1, inplace=True)
    return df

In [122]:
# compute the log loss
df_train = log_loss(df_train, "goal", "prediction")

# compute the mean log loss for each zone
mean_log_loss, x, y,  = py_help.prepare_heatmap(df_train, "posBeforeXMeters", "posBeforeYMeters", 24, 17,
                                                 agg_type="mean", agg_col="logLoss")

total_log_loss, _, _ = py_help.prepare_heatmap(df_train, "posBeforeXMeters", "posBeforeYMeters", 24, 17,
                                                 agg_type="sum", agg_col="logLoss")

# plot the heatmap
dict_info = {"Mean log loss": {"values": mean_log_loss, "display_type": ".3f"},
             "Total log loss": {"values": total_log_loss, "display_type": ".3f"}}

fig = py_help.create_heatmap(x, y, mean_log_loss, dict_info, title_name="Mean log loss")
fig.show()

Wow, that interesting, it seems like we are making the biggest error (on average) in the cells right in front of the goal... Let's have a closer look at what is going on by computing the overprediction per zone

In [123]:
# compute the number of shots per grid
nb_shots, x, y = py_help.prepare_heatmap(df_shots, "posBeforeXMeters", "posBeforeYMeters", 24, 17)

# mean prediction per cell
mean_prediction, _, _ = py_help.prepare_heatmap(df_train, "posBeforeXMeters", "posBeforeYMeters", 24, 17,
                                                 agg_type="mean", agg_col="prediction")

# mean target per cell
mean_target, _, _ = py_help.prepare_heatmap(df_train, "posBeforeXMeters", "posBeforeYMeters", 24, 17,
                                            agg_type="mean", agg_col="goal")

# compute overprediction for each cell
mean_overpred = mean_prediction - mean_target

# plot the heatmap
dict_info = {"Mean overprediction": {"values": mean_overpred, "display_type": ".3f"},
             "Mean target": {"values": mean_target, "display_type": ".3f"},
             "Mean prediction": {"values": mean_prediction, "display_type": ".3f"},
             "Number shots": {"values": nb_shots, "display_type": ".0f"}         
            }
fig = py_help.create_heatmap(x, y, mean_overpred, dict_info, title_name="Mean overprediction per zone")
fig.show()

Hm, it seems like we have quite some issues with correctly identifying the scoring probability for shots that are close to the goal line. While we severly underestimate the chance of scoring for shots taken directly in front of the goal (we predict 45%, while it is 63%), we significantly overestimate the chance when we are close to the goal line but not in front of the goal (we e.g. predict a chance of 20% while we only observe 5%). 

So how could we fix this issue and improve the model? It seems like we are severly overestimating when the angle to the goal post becomes too acute. So let's build a feature to measure the angle of the shot to the closest post.

In [124]:
def add_feature_angle(df):
    
    # distance to the post in y-direction (notice that the width of the goal is 7.32)
    df["dy"] = (df["distToCenter"] - 7.32 / 2)
    
    # if we have a negative angle (i.e. we are between the two posts right in front of the goal), we want to set the 
    # distance to 0
    df["dy"].clip(lower=0, inplace=True)
    
    # compute the angle to the closest post
    df["angle"] = df.apply(lambda row: math.degrees(math.atan2(row["dy"], row["distToGoalLine"])), axis=1)

    df.drop("dy", axis=1, inplace=True)
    return df

df_train = add_feature_angle(df_train)
df_test = add_feature_angle(df_test)

Nice, now we compute the angle for each shot. Let's quickly check if we did everything correctly by drawing a map with the angles per zone.

In [125]:
# mean prediction per cell
mean_angle, x, y = py_help.prepare_heatmap(df_train, "posBeforeXMeters", "posBeforeYMeters", 24, 17,
                                            agg_type="mean", agg_col="angle")

# plot the heatmap
dict_info = {"Mean angle": {"values": mean_angle, "display_type": ".3f"} 
            }

fig = py_help.create_heatmap(x, y, mean_angle, dict_info, title_name="Angle to goal post per zone")
fig.show()

Looks good! We indeed have the big angles within the zones that are close to the goal line and rather far from the post :-) Let's check how the scoring probabilities and overpredictions are for the different angles

In [126]:
df_train["angleBuckets"] = ml_help.build_buckets(df_train, "angle", 
                                                   step_size=5, 
                                                   max_val=70)

fig_prob, _ = ml_help.create_univariate_variable_graph(df=df_train, 
                                                  col="angleBuckets", 
                                                  target_col="goal", 
                                                  binned_cols=True,
                                                  title_name="Chance of scoring per angle")


fig_pred, _ = ml_help.create_univariate_variable_graph(df=df_train, 
                                                  col="angleBuckets", 
                                                  target_col="overprediction", 
                                                  binned_cols=True,
                                                  y2_axis_name="Overprediction (in pp)",
                                                  title_name="Overprediction for angles")

ml_help.combine_univariate_variable_graphs([fig_prob, fig_pred], 2, 1, shared_axis=False)

Cool, the right chart does look exactly as expected, doesn't it? We have huge overpredictions for the acute angles. The left chart on the other hand, looks a little bit odd. If you are directly in front of the goal, there is a higher chance of making it, while between 5 and 65 degrees it is almost constant. So, probably not a very good feature to add from a monotone perspective, is it? I mean we said before that we need to make sure that features are nicely monotone in the scoring probability and this one is not really nicely montone... 

There one nice thing about it: You remember when we said above that when *adding* features to an already existing model, we should check if we gain anything in respect of error reduction, i.e. look at the overprediction chart rather than the scoring probability chart. Well, the same holds true for the monotonicity! Instead of making sure that our model is monotone in the probabilities, we should make sure that it is monotone in the overprediction. And that does not look too bad, does it?

In [127]:
features = ["distToGoalLine", "distToCenter", "head/body", "strongFoot", "weakFoot", "counterAttack", 
            "corner", "smartPass", "duel", "shotBefore", "angle"]

# training of the logistic regression on the train set
reg_feat_2 = LogisticRegression(random_state=42)
reg_feat_2.fit(df_train[features], np.array(df_train["goal"]).ravel())

# prediction on the test set
pred_probs = reg_feat_2.predict_proba(df_test[features])[:,1]
print(f"Log loss on test set after event features: {sk_metrics.log_loss(df_test['goal'], pred_probs):.5f}")

Log loss on test set after event features: 0.27984


c:\programs_clemens\anaconda3\envs\soccer_analytics\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



*If you are getting a ConvergenceWarning here, do not worry. We will take care of this later on*

Nice, that improved our score quite a bit to 0.27984. And there is actually one tiny trick, how we can improve our results even more. You notice that in the above chart, it is basically constant up to 35 degrees and only then starts to increase, i.e. it is not (strictly) montone below 35? Let's just clip the values at 35 to have a strong montone curve.   

In [128]:
df_train["angleClip"] = df_train["angle"].clip(lower=35)
df_test["angleClip"] = df_test["angle"].clip(lower=35)

features = ["distToGoalLine", "distToCenter", "head/body", "strongFoot", "weakFoot", "counterAttack", 
            "corner", "smartPass", "duel", "shotBefore", "angleClip"]

# training of the logistic regression on the train set
reg_feat_2 = LogisticRegression(random_state=42)
reg_feat_2.fit(df_train[features], np.array(df_train["goal"]).ravel())

# prediction on the test set
pred_probs = reg_feat_2.predict_proba(df_test[features])[:,1]
print(f"Log loss on test set after event features: {sk_metrics.log_loss(df_test['goal'], pred_probs):.5f}")

Log loss on test set after event features: 0.27947


c:\programs_clemens\anaconda3\envs\soccer_analytics\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



Not bad, again a slight improve by just quickly clipping the values :-) 

Let's look at our overprediction map once again

In [129]:
df_train["prediction"] = reg_feat_2.predict_proba(df_train[features])[:,1]
df_train["overprediction"] = df_train["prediction"] - df_train["goal"]

In [130]:
# compute the number of shots per grid
nb_shots, x, y = py_help.prepare_heatmap(df_shots, "posBeforeXMeters", "posBeforeYMeters", 24, 17)

# mean prediction per cell
mean_prediction, _, _ = py_help.prepare_heatmap(df_train, "posBeforeXMeters", "posBeforeYMeters", 24, 17,
                                                 agg_type="mean", agg_col="prediction")

# mean target per cell
mean_target, _, _ = py_help.prepare_heatmap(df_train, "posBeforeXMeters", "posBeforeYMeters", 24, 17,
                                            agg_type="mean", agg_col="goal")

# compute overprediction for each cell
mean_overpred = mean_prediction - mean_target

# plot the heatmap
dict_info = {"Mean overprediction": {"values": mean_overpred, "display_type": ".3f"},
             "Mean target": {"values": mean_target, "display_type": ".3f"},
             "Mean prediction": {"values": mean_prediction, "display_type": ".3f"},
             "Number shots": {"values": nb_shots, "display_type": ".0f"}         
            }

# notice that we use the same scale as above
fig = py_help.create_heatmap(x, y, mean_overpred, dict_info, title_name="Mean overprediction per zone", 
                             colour_scale=(-0.15,0.15))
fig.show()

This looks way better now, doesn't it? Nevertheless, there is one thing that we should fix, namely the shots from very close which are right in front of the goal. Let's therefore build a binary feature that tells us, if the shot had an angle of 0 and was within 5 meters of the goal.

#### In front of goal

In [131]:
def add_feature_front_of_goal(df):
    df["frontOfGoal"] = 1*((df["distToGoalLine"] <= 5) & (df["angle"] == 0))
    return df

df_train = add_feature_front_of_goal(df_train)
df_test = add_feature_front_of_goal(df_test)

In [132]:
features = ["distToGoalLine", "distToCenter", "head/body", "strongFoot", "weakFoot", "counterAttack", 
            "corner", "smartPass", "duel", "shotBefore", "angleClip", "frontOfGoal"]

# training of the logistic regression on the train set
reg_feat_2 = LogisticRegression(random_state=42)
reg_feat_2.fit(df_train[features], np.array(df_train["goal"]).ravel())

# prediction on the test set
pred_probs = reg_feat_2.predict_proba(df_test[features])[:,1]
print(f"Log loss on test set after additional features: {sk_metrics.log_loss(df_test['goal'], pred_probs):.5f}")

Log loss on test set after additional features: 0.27865


c:\programs_clemens\anaconda3\envs\soccer_analytics\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



Amazing, this further improved our results and we are now down at a log loss of 0.27865 :-) And we are at an AUC of 79.4%...

Let's for a very last time have a look at the error map

In [133]:
df_train["prediction"] = reg_feat_2.predict_proba(df_train[features])[:,1]
df_train["overprediction"] = df_train["prediction"] - df_train["goal"]

In [134]:
# compute the number of shots per grid
nb_shots, x, y = py_help.prepare_heatmap(df_shots, "posBeforeXMeters", "posBeforeYMeters", 24, 17)

# mean prediction per cell
mean_prediction, _, _ = py_help.prepare_heatmap(df_train, "posBeforeXMeters", "posBeforeYMeters", 24, 17,
                                                 agg_type="mean", agg_col="prediction")

# mean target per cell
mean_target, _, _ = py_help.prepare_heatmap(df_train, "posBeforeXMeters", "posBeforeYMeters", 24, 17,
                                            agg_type="mean", agg_col="goal")

# compute overprediction for each cell
mean_overpred = mean_prediction - mean_target

# plot the heatmap
dict_info = {"Mean overprediction": {"values": mean_overpred, "display_type": ".3f"},
             "Mean target": {"values": mean_target, "display_type": ".3f"},
             "Mean prediction": {"values": mean_prediction, "display_type": ".3f"},
             "Number shots": {"values": nb_shots, "display_type": ".0f"}         
            }

# notice that we use the same scale as above
fig = py_help.create_heatmap(x, y, mean_overpred, dict_info, title_name="Mean overprediction per zone", 
                             colour_scale=(-0.15,0.15))
fig.show()

Doesn't that look nice now? :-)

#### Interaction terms

There is one last trick I want to show you, which sometimes comes in handy when looking for additional features. This trick is to create new features out of the existing ones by combining two or more features. 

Especially when looking at categorical features it sometimes happens that some other (most often continuous) feature behaves differently on the different categories of the feature. Let's look at an example to make a little bit clearer what I mean.

We are splitting the the body part by header and foot and look at the probabilites and overpredictions of the *distToGoalLine* measure separately.

In [135]:
df_header = df_train[df_train["head/body"] == 1].copy()
df_foot = df_train[df_train["head/body"] == 0].copy()

Let's start by looking at the shots done with the foot

In [136]:
df_foot["distToGoalLineBuckets"] = ml_help.build_buckets(df_foot, "distToGoalLine", 
                                                  step_size=2, 
                                                  min_val=2,
                                                  max_val=34)

fig_prob, _ = ml_help.create_univariate_variable_graph(df=df_foot, 
                                                  col="distToGoalLineBuckets", 
                                                  target_col="goal", 
                                                  binned_cols=True,
                                                  title_name="Chance of scoring for shots")


fig_pred, _ = ml_help.create_univariate_variable_graph(df=df_foot, 
                                                  col="distToGoalLineBuckets", 
                                                  target_col="overprediction", 
                                                  binned_cols=True,
                                                  y2_axis_name="Overprediction (in pp)",
                                                  title_name="Overprediction for shots")

ml_help.combine_univariate_variable_graphs([fig_prob, fig_pred], 2, 1, shared_axis=False)

Hm, that doesn't look too exciting, does it? The scoring proabability is monotone decreasing as expected and the error jumps between -1 and 1. So nothing to improve really. 

Let's now look at the same chart for headers

In [137]:
df_header["distToGoalLineBuckets"] = ml_help.build_buckets(df_header, "distToGoalLine", 
                                                  step_size=2, 
                                                  min_val=2,
                                                  max_val=14)

fig_prob, _ = ml_help.create_univariate_variable_graph(df=df_header, 
                                                  col="distToGoalLineBuckets", 
                                                  target_col="goal", 
                                                  binned_cols=True,
                                                  title_name="Chance of scoring for headers")


fig_pred, _ = ml_help.create_univariate_variable_graph(df=df_header, 
                                                  col="distToGoalLineBuckets", 
                                                  target_col="overprediction", 
                                                  binned_cols=True,
                                                  y2_axis_name="Overprediction (in pp)",
                                                  title_name="Overprediction for headers")

ml_help.combine_univariate_variable_graphs([fig_prob, fig_pred], 2, 1, shared_axis=False)

Cool, that looks way better in terms of improvement potential! :-) While we seem to underpredict the headers close to the goal line, we do overpredict the headers far from the goal. 

So why is this? Let me try to give an intuitive answer to the question. However, in case you really want to understand what is going on under the hood, I would recommend you to start reading about log odds and how logistic regression is a linear regression trying to predict those. So, what's going on here. What our model can do so far is to say that a header is always x% worse than a shot, i.e. the chance of scoring with a shot is 12% in a given situation and when I take the exact same situation but take a header instead of shooting the chance is reduced by 25%, i.e. it is 9%. And if we are in a situation where the shot scoring probability is 24%, the header probability is 18%. So far so good. Now, let's look at the scoring probabilities for shots done with the foot and for headers. We see that for both shots and headers the probability when being very close to the goal line is ~35%, meaning there is no real difference. When we look at a distance of 12-14 meters, however, the shooting probability is ~12% while the scoring probability is ~3%. So wait, we said that all the model can do so far is to say that a header is x% worse than a shot. However, when being close to the goal there is no difference and when being further away from the goal there is a huge difference... So what the model does is to take some average between no difference and huge difference. And that, you guessed it, leads to overestimating headers from long distance and underestimating headers from close to the goal, exactly what we are seeing above :-) (Caveat: As I said, the explanation above is intuitive and the percentage reduction in probabilities is mathematically not completely correct. So don't get too upset ;-))     

Nice, now we have understood why this is. But how can we fix this? There is actually a rather nice trick, which is to replace the binary *head/body* feature by the distance to the goal line, whenever the shot was a header, i.e. multiply the *head/body* feature with the *distToGoalLine* feature.

In [138]:
df_train["headDistToGoalLine"] = df_train["head/body"] * df_train["distToGoalLine"]
df_test["headDistToGoalLine"] = df_test["head/body"] * df_test["distToGoalLine"]

In [139]:
# I'll leave the head/body feature in for now as we will look at correlations in the next section
features = ["distToGoalLine", "distToCenter", "head/body", "strongFoot", "weakFoot", "counterAttack", 
            "corner", "smartPass", "duel", "shotBefore", "angleClip", "frontOfGoal", "headDistToGoalLine"]

# training of the logistic regression on the train set
reg_feat_3 = LogisticRegression(random_state=42)
reg_feat_3.fit(df_train[features], np.array(df_train["goal"]).ravel())

# prediction on the test set
pred_probs = reg_feat_3.predict_proba(df_test[features])[:,1]
print(f"Log loss on test set after interaction term: {sk_metrics.log_loss(df_test['goal'], pred_probs):.5f}")
print(f"AUC on test set after after interaction term: {sk_metrics.roc_auc_score(df_test['goal'], pred_probs)*100:.2f}%")

Log loss on test set after interaction term: 0.27829
AUC on test set after after interaction term: 79.45%


c:\programs_clemens\anaconda3\envs\soccer_analytics\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



Hurray, we again reduced our loss and are now at 0.27829. And our AUC is ~79.45%, not bad...

Let's quickly again look at the overprediction of headers

In [140]:
df_train["prediction"] = reg_feat_3.predict_proba(df_train[features])[:,1]
df_train["overprediction"] = df_train["prediction"] - df_train["goal"]

df_header = df_train[df_train["head/body"] == 1].copy()

df_header["distToGoalLineBuckets"] = ml_help.build_buckets(df_header, "distToGoalLine", 
                                                  step_size=2, 
                                                  min_val=2,
                                                  max_val=14)

fig_prob, _ = ml_help.create_univariate_variable_graph(df=df_header, 
                                                  col="distToGoalLineBuckets", 
                                                  target_col="goal", 
                                                  binned_cols=True,
                                                  title_name="Chance of scoring for headers")


fig_pred, _ = ml_help.create_univariate_variable_graph(df=df_header, 
                                                  col="distToGoalLineBuckets", 
                                                  target_col="overprediction", 
                                                  binned_cols=True,
                                                  y2_axis_name="Overprediction (in pp)",
                                                  title_name="Overprediction for headers")

ml_help.combine_univariate_variable_graphs([fig_prob, fig_pred], 2, 1, shared_axis=False)

This looks way better! Also focus on the scale of the right graph. We are now at -2 to 1.5 while we were at -4 to 3 before the introduction of our new feature!

<a id="multivariate"></a>

# 6. Multivariate analysis

So far we always only cared about one variable at a time but did not really look at all the variables together. This is what we want to do in this section. 

In a nutshell, multivariate analysis is all about finding highly correlated features and thinking about how to deal with them. This needs to be done for two reasons:

1. Highly correlated features might lead to overfitting and actually worsen your results
2. In my opinion, most often more importantly, highly correlated features can very easily mess with the feature interpretation and importance

So, let's get started by calculating the correlations between the features

In [141]:
features = ["distToGoalLine", "distToCenter", "head/body", "strongFoot", "weakFoot", "counterAttack", 
            "corner", "smartPass", "duel", "shotBefore", "angleClip", "frontOfGoal", "headDistToGoalLine"]

# we use a helper function as this is a little bit more flexible than the *normal* correlation matrix. We will learn more
# about this in the next notebook
df_correls = ml_help.associations(df_train[features], return_results=True)
df_correls

,distToGoalLine,distToCenter,head/body,strongFoot,weakFoot,counterAttack,corner,smartPass,duel,shotBefore,angleClip,frontOfGoal,headDistToGoalLine
distToGoalLine,1.000000,0.155511,-0.356602,0.308410,-0.041238,-0.012840,-0.086709,-0.126956,-0.094662,-0.121651,-0.327432,-0.204521,-0.221742
distToCenter,0.155511,1.000000,-0.270654,0.203692,0.005724,0.039487,-0.037323,0.038862,0.026022,-0.064055,0.378798,-0.159355,-0.224659
head/body,-0.356602,-0.270654,1.000000,-0.596924,-0.210813,-0.083461,0.170690,-0.082934,0.178192,-0.027848,-0.060158,0.083156,0.885736
strongFoot,0.308410,0.203692,-0.596924,1.000000,-0.658428,0.047331,-0.089364,0.043028,-0.162540,-0.005255,0.017783,-0.063973,-0.528717
weakFoot,-0.041238,0.005724,-0.210813,-0.658428,1.000000,0.020627,-0.051244,0.025375,0.030875,0.032526,0.034768,-0.000065,-0.186725
counterAttack,-0.012840,0.039487,-0.083461,0.047331,0.020627,1.000000,-0.037081,0.066806,0.002620,-0.029613,0.019606,-0.026301,-0.073602
corner,-0.086709,-0.037323,0.170690,-0.089364,-0.051244,-0.037081,1.000000,-0.033042,-0.119662,-0.022962,0.014484,0.009967,0.138894
smartPass,-0.126956,0.038862,-0.082934,0.043028,0.025375,0.066806,-0.033042,1.000000,-0.161310,-0.030954,0.099062,-0.014900,-0.072887
duel,-0.094662,0.026022,0.178192,-0.162540,0.030875,0.002620,-0.119662,-0.161310,1.000000,-0.112099,0.034777,-0.019721,0.156558
shotBefore,-0.121651,-0.064055,-0.027848,-0.005255,0.032526,-0.029613,-0.022962,-0.030954,-0.112099,1.000000,0.018274,0.114556,-0.032765


Ok, that is pretty hard to deal with. Let's quickly draw a heatmap to get it in a nicer format

In [142]:
# get the absolute values of the correlations
correls = np.abs(df_correls.values)

# we set the diagonal to 0 to not disturb the picture
np.fill_diagonal(correls, 0)

# draw a heatmap
fig = px.imshow(correls,
                labels=dict(color="Correlation"),
                x=df_correls.columns,
                y=df_correls.columns,
                zmax=1,
                zmin=0,
                color_continuous_scale=["green", "red"]
               )
fig.update_xaxes(side="top")
fig.show()

Ok, that is way easier to look at. So, obviously we only need to look at the red points in the heatmap, as those are the combinations with high correlation (even though some people will probably kill me, I would argue that correlations below 50% do most often not cause severe damage, so we will not look at those):

1. Let's start by looking at the "correlation diamond" in the top left. We see that *strongFoot* has a high correlation with both *head/body* and *weakFoot*. You remember how we created those variables? We had one categorical column saying whether the shot was done with the strong foot, the weak foot or the head and then made 3 dummy columns out of it. So let's assume I tell you that a shot was neither made with the head nor with the weak foot. Then you obviously immediately know that it was made with the strong foot. So, no surprise that we see some correlation here. Even more generally: <b> When dummy encoding a categorical column, you should always delete one of the created columns.</b> As a rule of thumb, I would always go with the column of the value that is most common, as this will have the biggest correlation to the other columns. So let's delete the *strongFoot* column in our case.


2. There is one more highly correlated feature pair, which is *head/body* and *headDistToGoalLine*. Again, the way we created the feature it is absolutely no surprise that there is a rather high correlation. But as we said, we would like to replace *head/body* by *headDistToGoalLine*. So let's do this and take out the *head/body* column

Nice, let's see how the correlation matrix looks now

In [143]:
# features w/o *head/body* and *strongFoot*
features = ["distToGoalLine", "distToCenter", "weakFoot", "counterAttack", 
            "corner", "smartPass", "duel", "shotBefore", "angleClip", "frontOfGoal", "headDistToGoalLine"]

# we use a helper function as this is a little bit more flexible than the *normal* correlation matrix. We will learn more
# about this in the next notebook
df_correls = ml_help.associations(df_train[features], return_results=True)

# get the absolute values of the correlations
correls = np.abs(df_correls.values)

# we set the diagonal to 0 to not disturb the picture
np.fill_diagonal(correls, 0)

# draw a heatmap
fig = px.imshow(correls,
                labels=dict(color="Correlation"),
                x=df_correls.columns,
                y=df_correls.columns,
                zmax=1,
                zmin=0,
                color_continuous_scale=["green", "red"]
               )
fig.update_xaxes(side="top")
fig.show()

Very nice, all correlations are below 0.5 now, so we are done with the eliminating correlated features. But wait, now that we have less features, did the performance also drop? Let's quickly check...

In [144]:
# I'll leave the head/body feature in for now as we will look at correlations in the next section
features = ["distToGoalLine", "distToCenter", "weakFoot", "counterAttack", 
            "corner", "smartPass", "duel", "shotBefore", "angleClip", "frontOfGoal", "headDistToGoalLine"]

# training of the logistic regression on the train set
reg_multi = LogisticRegression(random_state=42)
reg_multi.fit(df_train[features], np.array(df_train["goal"]).ravel())

# prediction on the test set
pred_probs = reg_multi.predict_proba(df_test[features])[:,1]
print(f"Log loss on test set after interaction term: {sk_metrics.log_loss(df_test['goal'], pred_probs):.5f}")
print(f"AUC on test set after after interaction term: {sk_metrics.roc_auc_score(df_test['goal'], pred_probs)*100:.2f}%")

Log loss on test set after interaction term: 0.27807
AUC on test set after after interaction term: 79.48%


c:\programs_clemens\anaconda3\envs\soccer_analytics\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



Whut?! We deleted columns but both the log loss and the AUC got better?! While this may look surprising at the beginning, trust me that you will pretty quickly learn that more features it not always better ;-) 

<a id="final"></a>

# 7. Building the final model

#### Standard scaling

Congrats, you have almost made it to the end of this notebook and have come a long way in creating an expected goal model using a logistic regression! In this section we are finally going to build our final model :-)

Actually, there is only one small step left that we need to do for two reasons:

1. Get rid of the ConvergenceWarning in every training step (I guess you also get the warning, don't you ;-))
2. Tweak the model slightly so that we can interpret the influence of the variables in an easier way

The step that we need to do, is to make different variables "live" on the same scale. We can achieve that by normalizing for mean and standard deviation. You see, what we currently have is that
- *distToGoalLine* takes values between 0 and 105
- *angle_clip* takes values between 0 and 90
- *corner* takes values 0 and 1 etc.

This makes it very hard to compare different variables and also for the model to optimize. We are therefore making sure that all of the features take values on a comparable scale. 

<b>Important: All scaling needs to be done only looking at the training dataset!</b>

In [145]:
# same features as before
features = ["distToGoalLine", "distToCenter", "weakFoot", "counterAttack", "corner", "smartPass", "duel", "shotBefore", 
            "angleClip", "frontOfGoal", "headDistToGoalLine"]

# copy the features in a new data frame
feat_train = df_train[features].copy()
feat_test = df_test[features].copy()

feature_measures = dict()

# normalize the features - make sure you take the mean and standard deviation both from the training dataset! 
for feat in features:
    
    mean_feat = feat_train[feat].mean()
    std_feat = feat_train[feat].std()
    
    # save the mean and standard deviation for later usage
    feature_measures[feat] = {"mean": mean_feat, "std": std_feat}
    
    feat_train[feat] = (feat_train[feat] - mean_feat) / std_feat
    feat_test[feat] = (feat_test[feat] - mean_feat) / std_feat

# training of the logistic regression on the train set
reg_final = LogisticRegression(random_state=42)
reg_final.fit(feat_train, np.array(df_train["goal"]).ravel())

# save model for future use
# on purpose I commented this out to not overwrite the model we developed together. If you want to overwrite and
# save your model, you can bring it back in again
#io.save_model(reg_final, "expected_goals_logreg")

# prediction on the test set
pred_probs = reg_final.predict_proba(feat_test)[:,1]
print(f"Log loss on test set: {sk_metrics.log_loss(df_test['goal'], pred_probs):.5f}")
print(f"AUC on test set: {sk_metrics.roc_auc_score(df_test['goal'], pred_probs)*100:.2f}%")

# save the predictions in the training and test set
df_train["prediction"] = reg_final.predict_proba(feat_train)[:,1]
df_train["overprediction"] = df_train["prediction"] - df_train["goal"]

df_test["prediction"] = pred_probs
df_test["overprediction"] = df_test["prediction"] - df_test["goal"]

Log loss on test set: 0.27810
AUC on test set: 79.48%


Great, we got rid of the warning and do still have the same log loss as before! :-)

#### Calibration plot

You remember what our initial goal was? Why we built the model in the beginning? Correctly, we wanted to attach the probability to score a goal to each shot. Then we started talking about log loss, built features and checked for correlations etc. But did we reach our initial goal? Is our model able to predict the probabilities correctly? 

We can check this by looking at so-called calibration plots. The idea behind these plots is the following: You put all of your prediction probabilities in buckets. Let's say bucket 1 are all predictions of 0% - 5%, bucket 2 are all predictions 5% - 10% etc. Then you go ahead and compute for each of the buckets the *observed* scoring likelihood. If the observed likelihood is between those boundaries (i.e. between 0% and 5% for bucket 1) your model is well-calibrated and you are ready to use it. Makes sense? 

<b>Important:</b> You should <b>always</b> look at calibration plots when predicting probabilities. While this is most likely no big problem for logistic regression (as we will see below) very weird things can happen for almost all other classification models! The nasty part is this: While all models have a *predict_proba* function, for most of the models it does <b>not</b> necessarily return the probability. You can read more about this [here](https://scikit-learn.org/stable/modules/calibration.html).

Luckily, creating calibration plots is quite simple with the functions we already have

In [91]:
df_train["predictionBucket"] = ml_help.build_buckets(df_train, 
                                             col="prediction", 
                                             step_size=0.05)

# convert to percent and integer
df_train["predictionBucket"] *= 100
df_train["predictionBucket"] = df_train["predictionBucket"].astype("int")

fig_prob, _ = ml_help.create_univariate_variable_graph(df=df_train, 
                                                       col="predictionBucket", 
                                                       target_col="goal", 
                                                       binned_cols=True,
                                                       title_name="Calibration plot")
fig_prob.show()

Ok, the plot looks a little bit funny for very high prediction. E.g. the probability when predicting a value between 75% and 80% is "only" 65.38% in reality. However, when looking at the blue bars, we notice that we basically do not have any observations there. So let's focus on the more common ones and maybe be a little bit more specific there.

In [92]:
df_train["predictionBucket"] = ml_help.build_buckets(df_train, 
                                             col="prediction", 
                                             step_size=0.04,
                                             max_val=0.44,
                                             delete=True)

df_train["predictionBucket"] *= 100

df_relevant = df_train[df_train["predictionBucket"].notnull()].copy()
df_relevant["predictionBucket"] = df_relevant["predictionBucket"].astype("int")

fig_prob, _ = ml_help.create_univariate_variable_graph(df=df_relevant, 
                                                       col="predictionBucket", 
                                                       target_col="goal", 
                                                       binned_cols=True,
                                                       title_name="Calibration plot")
fig_prob.show()

That does look very nice! Two great things about our model:

1. It is very well calibrated. When hovering over the orange line you will notice that almost every point is within the bucket it should be! :-)
2. It is great to see that the range of predictions is actually quite wide! While we have shots with probabilities < 4%, we still have quite a few of them with probability > 40%

Again, please always check the calibration plot after model development. If you use e.g. a random forest as your model, there is a very high chance that your plot is not going to look as nicely as this one!

#### Model performance

Ok, so we are feeling quite happy our model. But how does it compare to other models that have been developed. If search for "expected goal model", the first link that pops up (at least for me) is an [article](http://business-analytic.co.uk/blog/evaluating-expected-goals-models/) comparing different expected goal models. The model that is most comparable to ours (from a feature perspective) is the *Standard model* which has an AUC of 78.7%. 

There is one big difference, however, which is that in their model both penalties and free kicks are included. And we currently  do not have them included here. So let's quickly build two very rudimentary models for penalties and free kicks to make numbers more comparable.  

In [93]:
# penalty model consists of giving the same probability to all penalties
df_penalty = df_events[df_events["subEventName"] == "Penalty"].copy()
df_train_penalty, df_test_penalty, _, _ = train_test_split(df_penalty, df_penalty["goal"], test_size=0.25, random_state=42)
df_train_penalty = df_train_penalty.copy()
df_test_penalty = df_test_penalty.copy()

df_train_penalty["prediction"] = df_train_penalty["goal"].mean()
df_test_penalty["prediction"] = df_train_penalty["goal"].mean()

print(f"Scoring likelihood of {df_train_penalty['goal'].mean() * 100:.2f} for all penalties")

Scoring likelihood of 74.32 for all penalties


In [94]:
# free kick model uses position as the only feature
df_free_kick = df_events[df_events["subEventName"] == "Free kick shot"].copy()

df_train_free_kick, df_test_free_kick, _, _ = train_test_split(df_free_kick, df_free_kick["goal"], 
                                                               test_size=0.25, random_state=42)

df_train_free_kick = transformation_univariate(df_train_free_kick)
df_test_free_kick = transformation_univariate(df_test_free_kick)

features = ["distToGoalLine", "distToCenter"]
reg_fk = LogisticRegression(random_state=42)
reg_fk.fit(df_train_free_kick[features], np.array(df_train_free_kick["goal"]).ravel())

# prediction on the test set
df_test_free_kick["prediction"] = reg_fk.predict_proba(df_test_free_kick[features])[:,1]

We compute the combined AUC of all three models

In [95]:
all_preds = list(pred_probs) + list(df_test_penalty["prediction"]) + list(df_test_free_kick["prediction"])
all_true = list(df_test["goal"]) + list(df_test_penalty["goal"]) + list(df_test_free_kick["goal"])

print(f"Combined AUC including penalties and free kicks: {sk_metrics.roc_auc_score(all_true, all_preds)*100:.2f}%")

Combined AUC including penalties and free kicks: 80.18%


Our AUC including free kicks and penalties is at ~80.2%. Not so bad, is it?! While I do not want to say at all that our model is better than the one presented there (slightly different features, different data source, ...), I guess we can definitely say that we built a decent model in predicting expected goals :-)

<a id="feature_interpretation"></a>

# 8. Feature interpretation

Analyzing the features is a very important step in the process of model development for at least two reasons:

1. It allows you to communicate why the model predicted what it predicted
2. It is often a cause to trigger change in the actual doing (assume our model identifies that there is a very severe disadvantage when shooting with the weak foot; this might trigger to slightly adjust the training and add more elements of shooting with the weak foot) 
3. It helps you to sense-check the model results and identify potential bugs ;-)

This section is again split into smaller parts in which we answer different questions

#### Does a variable have a positive or a negative effect on the likelihood to score?

This question is fortunately somehow simple to answer. All you need to do is to look at the signs of the coefficients of our final model. So let's print all of them quickly:

In [96]:
features = ["distToGoalLine", "distToCenter", "weakFoot", "counterAttack", "corner", "smartPass", 
            "duel", "shotBefore", "angleClip", "frontOfGoal", "headDistToGoalLine"]

print("Coefficients")
print("############")
for i, coef in enumerate(reg_final.coef_[0]):
    print(f"{features[i]}: {coef:.3f}")

Coefficients
############
distToGoalLine: -1.195
distToCenter: -0.467
weakFoot: -0.032
counterAttack: 0.114
corner: -0.091
smartPass: 0.080
duel: -0.069
shotBefore: 0.080
angleClip: -0.304
frontOfGoal: 0.090
headDistToGoalLine: -0.381


Even though the magnitude of the number might be of some interest as well, let's for now only focus on the sign, i.e. is the coefficient positive or negative: A negative coefficient means that the bigger the value in the feature, the more unlikely the player becomes to score when shooting *given that all other features stay the same*. A positive coefficient, in constrast, says that the bigger the value in the feature, the more likely the player is to score. 

Let's look at *distToGoalLine*: There we have a negative number, indicating that the bigger the distance to the goal line, the more unlikely it is for a shot to find its way into the net. Makes sense, doesn't it? So does that mean that every shot taken from 15 meters has a lower probability than a shot taken from 12 meters? No, it doesn't, that's what we mean with *given all other features stay the same*. What the negative coefficient says, is, that if I have two shots, both with the weak foot, both after a counter attack, no corner, no smart pass, no duel and no other shot before that were taken right from the center and one of them was from 15 meters and the other from 12 meters, then the one taken from 12 meters had a higher chance to score. That is something you should always keep in mind.

We can now go through some of features and have a look at whether the sign of the coefficient does intuitively make sense:

- <b>*distToCenter*</b>: The further we are from the center of the field the less likely we score. Makes sense :-)


- <b>*weakFoot*</b>: Shooting with the weak foot lowers the probability to score. But wait, lower from what? In comparison to a header? Highly doubt it... You remember when we said above, that out of the variables *strongFoot*, *weakFoot* and *head/body* we were going to delete the *strongFoot*. Well, the variable that was deleted is always the baseline and the one we should compare the coefficient to. In our case, therefore, we should say: Shooting with the weak foot lowers the probability to score compared to the strong foot. That sounds reasonable, doesn't it?


- <b>*counterAttack*</b>: If someone shoots as part of a counter attack, there is a higher chance to score. This is probably due to the fact that the opponents defense is not as organized yet. Notice: You might be tempted to say: "Well, of course there is a higher chance as in a counter attack the attacker stands often closer to the goal". But again, the positive sign tells us that there is a higher likelihood to score after a counter attack <b>if</b> all other variables stays the same, i.e. if both shots are taken from the exact same position.


- <b>*corner*</b>, <b>*smartPass*</b>, <b>*duel*</b>, <b>*shotBefore*</b> and <b>*angleClip*</b> are pretty clear, aren't they? So let's skip them.


- <b>*frontOfGoalLine*</b>: That's an interesting one. You remember what it was? It is a binary variable that tells us if the player stands right in front of the goal. The coefficient is positive which makes sense. However, notice that there is some link between the *frontOfGoalLine* variable and e.g. the *distanceToGoalLine* variable. If we are in front of the goal we are obviously also close to the goal. And we learned above, that the *distanceToGoalLine* already tells us that it is good to be close to the goal. So what does the positive sign in *frontOfGoalLine* add to that? Think of it as an multiplier of the scoring likelihood, i.e. while the *distanceToGoalLine* already says that it is good to be in front of the goal, the *frontOfGoalLine* adds to this, telling us that is extremely good to be in front of the goal. 


- <b>*headDistToGoalLine*</b>: This one is even more tricky... By just looking at it, it says that the further we are from the goal when taking a header, the less likely we score. Ok, sure, makes sense, but didn't we already say above that this is true in general? Again, think of this variable as a multiplier in case of a header. While it is in general bad to be far from the goal, it is especially bad when taking a header.

Nice, now we understand for all of the features whether they have a positive or negative effect on scoring. 

#### What are the most important features?

Loosely speaking, you can get the importance of a feature by looking at the magnitude of the coefficient. So let's quickly print them again, but now focus on the (absolute) magnitude rather than the sign. Notice: This is only valid after we have done the scaling of the variables!

In [97]:
features = ["distToGoalLine", "distToCenter", "weakFoot", "counterAttack", "corner", "smartPass", 
            "duel", "shotBefore", "angleClip", "frontOfGoal", "headDistToGoalLine"]

print("Coefficients")
print("############")
for i, coef in enumerate(reg_final.coef_[0]):
    print(f"{features[i]}: {coef:.3f}")

Coefficients
############
distToGoalLine: -1.195
distToCenter: -0.467
weakFoot: -0.032
counterAttack: 0.114
corner: -0.091
smartPass: 0.080
duel: -0.069
shotBefore: 0.080
angleClip: -0.304
frontOfGoal: 0.090
headDistToGoalLine: -0.381


The biggest (absolute) coefficient belongs to *distToGoalLine*, indicating that the distance to the goal line is the most important feature. That makes sense! The next one is *distToCenter*, followed by *headDistToGoalLine* and *angleClip*. Wait, don't they all measure the position on the field? Actually, they do! And yes, they are somehow dependent on each other. Only if *distToCenter* exceeds a certain size, is it possible to have *angleClip* of bigger than 0. Or even worse, for headers we do see same values in *headDistToGoalLine* and *distToGoalLine*. So should we just go to the coach and tell him the most important variables based on the coefficients? 

In my opinion, we shouldn't, but we should try to make sense of it in a way, that we can explain it to a non data scientist! So what is really the message here? 
1. Position on the field is extremely important when shooting (obvious, but still)
2. The distance to the goal line is super important, and even more so when taking a header
3. The further you are from the center of the field, the more unlikely to score. And that is especially true when you are close to the goal line (i.e. the angle gets very small)

So, instead of just putting the coefficients on a slide and telling the coach the bigger the number, the more important the feature, we should really try to make sense of the output and try to translate it into a more digestible language. 

#### So, how exactly do the scoring probabilities change when changing a feature?

So, we already know whether a feature increases or decreases the change of scoring and which features are the most important ones. The natural next question is: How much does each feature change the scoring probability, i.e. if an average player takes shot with his strong foot from 12 meters instead of 15 meters, what is the difference in the scoring probability?

Unfortunately, this question is not that easy to answer anymore. You might have heard, that interpreting coefficients in a logistic regression is simple as the logistic regression is linear. What a lot of people don't tell you, however, is, that it is not linear in the probabilities but linear in the log odds (you might have heard about [odds](https://en.wikipedia.org/wiki/Odds) in betting or gambling before). So in order to answer the question above one would have to:
1. Scale the 12 meters and 15 meters as we did above
2. Take the difference between the scaled values and multiply with the coefficient of *distToGoalLine*
3. Take this value and throw it into the exponential function to get the percentage difference in odds
4. Rescale to probability difference considering the relationship between odds and probabilities

Isn't that easy? :-) And having said this, it obviously becomes even slightly more difficult when e.g. looking for the difference for a header between 4m and 6m...

Ok, so getting the probability change directly from the coefficients does not seem to be a good option. What can we do instead? Let's see, we have a model that predicts probabilities given some input. So, wouldn't it be easier to just use the model, compute the probabilities for both 12 meters and 15 meters given the other context of the shot and then check the difference in probabilities? Let's do exactly this.

We first need to build a data frame for the two shots we are interested in

In [127]:
# Set some assumptions about the shot (other than the distance)
dist_center = 0
counter_attack = 0
duel = 0
body_part = "strongFoot"
corner = 0
smart_pass = 0
shot_before = 0

# build a data frame with the two shots
df_shots = pd.DataFrame()
df_shots["distToGoalLine"] = [12, 15] # we want to know the probabilities for 12 and 15 meters
df_shots["distToCenter"] = dist_center
df_shots["duel"] = duel
df_shots["corner"] = corner
df_shots["smartPass"] = smart_pass
df_shots["shotBefore"] = shot_before
df_shots["counterAttack"] = counter_attack

for col in ["head/body", "strongFoot", "weakFoot"]:
    df_shots[col] = 1*(col == body_part)

df_shots.head()

,distToGoalLine,distToCenter,duel,corner,smartPass,shotBefore,counterAttack,head/body,strongFoot,weakFoot
0,12,0,0,0,0,0,0,0,1,0
1,15,0,0,0,0,0,0,0,1,0


And now we can just throw it into our model. Notice that we use some class *ExpectedGoalModel* to do this for us. This is nothing else than all the steps we did above combined in one class, i.e. feature transformation, feature creation and feature scaling. This just makes it way easier to use.

In [143]:
egm = goal_model.ExpectedGoalModelLogistic(debug_mode=False)

# create the features we have not set yet like e.g. angle or headerDistToGoalLine
df_features = egm.create_features(df_shots, overwrite=False)

# normalize the features as done above
df_features = egm.normalize_features(df_features)

# use our final model to make the prediction
egm.predict_proba(df_features)[:,1]

array([0.23407369, 0.16712966])

Nice, so we know that the probability to score fell from 23.4% to 16.7% when shooting from 15 meters instead of 12 meters (and considering all the other features that we had set above). Let's take this one step further and see how the complete field looks like when taking a shot with the strong foot :-)

In [100]:
# build a data frame with all positions
pos_x = np.arange(106)
pos_y = np.arange(69)

df_shots = pd.DataFrame([(x, y) for x in pos_x for y in pos_y], columns=["posBeforeXMeters", "posBeforeYMeters"])
df_shots["duel"] = duel
df_shots["corner"] = corner
df_shots["smartPass"] = smart_pass
df_shots["shotBefore"] = shot_before
df_shots["counterAttack"] = counter_attack

for col in ["head/body", "strongFoot", "weakFoot"]:
    df_shots[col] = 1*(col == body_part)

# create the features we have not set yet like e.g. angle or headerDistToGoalLine
df_features = egm.create_features(df_shots, overwrite=False)

# normalize the features as done above
df_features = egm.normalize_features(df_features)

# use our final model to make the prediction
df_shots["prediction"] = egm.predict_proba(df_features)[:,1] * 100

# compute the number of shots per grid
score_prob, x, y = py_help.prepare_heatmap(df_shots, "posBeforeXMeters", "posBeforeYMeters", 36, 25, 
                                           agg_type="mean", agg_col="prediction")

# plot the heatmap
dict_info = {"Scoring probability": {"values": score_prob, "display_type": ".2f"}       
            }

# notice that we use the same scale as above
fig = py_help.create_heatmap(x, y, score_prob, dict_info, title_name="Scoring probability when shooting with strong foot", 
                             colour_scale=(0,35))
fig.show()

That looks good and makes a lot of sense, doesn't it? Just out of interest, let's do the same thing for a header now

In [101]:
body_part = "head/body"

for col in ["head/body", "strongFoot", "weakFoot"]:
    df_shots[col] = 1*(col == body_part)

# create the features we have not set yet like e.g. angle or headerDistToGoalLine
df_features = egm.create_features(df_shots, overwrite=False)

# normalize the features as done above
df_features = egm.normalize_features(df_features)

# use our final model to make the prediction
df_shots["prediction"] = reg_final.predict_proba(df_features)[:,1] * 100

# compute the number of shots per grid
score_prob, x, y = py_help.prepare_heatmap(df_shots, "posBeforeXMeters", "posBeforeYMeters", 36, 25, 
                                           agg_type="mean", agg_col="prediction")

# plot the heatmap
dict_info = {"Scoring probability": {"values": score_prob, "display_type": ".2f"}       
            }

# notice that we use the same scale as above
fig = py_help.create_heatmap(x, y, score_prob, dict_info, title_name="Scoring probability for headers", 
                             colour_scale=(0,35))
fig.show()

Nice! Exactly the way we had expected it! And when hovering over the different cells, one can nicely see that the values get smaller with each cell further away from the goal (remember that as we will see differently in the next notebook ;-)) And isn't it nicer to answer the difference in scoring probabilities in this way rather than trying to solve some odds to probability relationships? 

<a id="model_usage"></a>

# 9. Model usage

You are almost done, I promise! :-) There is just one more thing I would like to highlight when building and using models. In most of the real-world examples building the best model in the sense of log loss, AUC, ... is not the ultimate goal, but using the model to answer some question is. 

Let's take our expected goal model: We want to use it to identify
- Which team should have won a match based on the chances they had?
- Which striker significantly outperforms his expected goals and is therefore "better" than the average striker?
- Which goalie gets significantly less goals than he should and is therefore most probably a good goalie on the line?

So, we use our model as a <b>tool</b> to in the end answer some question which is completely different from "what is the chance of scoring when shooting". Now, what is the problem with tools? If I ask you which tool is better, a hammer or screwdriver, you will immediately respond: "It depends...". Well, the same is true for our model. 

Let's look at an example to hopefully make things a little bit clearer. Namely, let's look at a feature that tells us whether the shooting team leads the game (before the shot) or not.

In [102]:
# get the current (relative) score, i.e. leads by 1 goal, 2 goals, etc. for each shot
df_standing = ed_help.compute_current_standing(df_events)
df_train = pd.merge(df_train, df_standing[["id", "currentRelativeScore"]], on="id")

# cut at leading or loosing by 3 goals
df_train["currentRelativeScore"] = df_train["currentRelativeScore"].clip(lower=-3, upper=3)

# show scoring probabilities and overpredictions
fig_prob, _ = ml_help.create_univariate_variable_graph(df=df_train, 
                                                  col="currentRelativeScore", 
                                                  target_col="goal", 
                                                  title_name="Chance of scoring with relative score")


fig_pred, _ = ml_help.create_univariate_variable_graph(df=df_train, 
                                                  col="currentRelativeScore", 
                                                  target_col="overprediction", 
                                                  y2_axis_name="Overprediction (in pp)",
                                                  title_name="Overprediction with relative score")

ml_help.combine_univariate_variable_graphs([fig_prob, fig_pred], 2, 1, shared_axis=False)

Ok, so two observations:

1. Apparently, a team that is leading has a higher chance of scoring when shooting. There might be several reasons for this: 
    - Players take less desperate shots where they actually still have a defender in front of them
    - Players might have more confidence
    - Teams tend to pass longer until they find someone in a very good position
    - Better strikers play in the teams that lead more often
    
    
2. We are currently significantly underestimating shots that are done by teams that are leading, i.e. we could most likely improve our model by incorporating this feature

Question is: Should we incorporate this feature into the model? Unfortunately, the answer is the same as with the hammer and the screwdriver: It depends... If we want to build a model that
- has the best possible log loss as we win some money if we find the model with the lowest loss. Yes, then of course it makes sense to incorporate this feature.


- predicts whether it was fair that our team lost the game. No, in this case we most definitely do not want to incorporate the feature. Why? Imagine the following conversation with the coach: 
    - "I'm not sure whether it was fair that we lost the game." 
    - "I guess it was, as my model tells me that the opposite team had the better chances."
    - "Ok, so why does the model tell you this?"
    - "Because they were leading..."
    
  You see how this conversation might be difficult from here on :-)
  
  
- build a model to identify good strikers who outperform their peers. Uff, that's a tough one and cannot be answered without better understanding *why* the team that is leading has a higher chance of scoring. If it is one of the first 3 points mentioned above, then yes, you might want to think about incorporating it. If it's the last one, then most definitely no, you do not want to add it.

So, to summarize the above: <b> When building a model always have in mind why you build it and do not blindly try to optimize some loss function. And vice versa: When using a model, make sure that it actually fits your problem.</b>

While this might sound obvious, trust me when I tell you that there are a ton of people out there that do not follow this ;-)

# Summary

Amazing, you made it, congrats!!! :-) Wow, that was a long journey to get here, but I hope you still enjoyed it and learned quite a lot while going through it! I promise that if you have understood the topics above you are on a very good way to becoming a very decent data scientist :-) 

So, let's quickly summarize what we learned and how you might go through a modelling process if you are going to build your own model:

1. <b>Think about why you want to build the model.</b> Is it to only predict the outcome as good as possible or do you want to use it as a tool to get some insights out of it? This has some big impact on how you build your model.
2. <b>Understand the underlying data.</b> Is there any bias in the data? How does this affect the analysis you want to make. What conclusions can you make and which not? 
3. <b>Define an appropriate metric.</b> What do you really want to predict? Probabilities, right/wrong, the ordering, ... Answer this in order to find an appropriate metric to measure a model's goodness.
4. <b>Detect and treat outliers.</b> Make sure you look and outliers and think about how to treat them.
5. <b>Define training and test set.</b> Split your data into a training and a test set. Make sure you keep them separate and never incorporate information from the test set into your training set.
6. <b>Build initial, rudimentary baseline model.</b> Build a first rudimentary model by e.g. just using features that are immediately there without a lot of feature engineering effort. For logistic regression, you might still want to make sure to make sure that features are monotone in the target (for continuous features) and dummy encoded (for categorical features). 
7. <b>Improve your initial model.</b> Look for features that capture things the other features have not captured yet, i.e. focus on error reduction. Make sure that features are montone in the (signed) error. Look at examples or plots to find out where the biggest errors happen in your current model. Think about interactions if it makes sense. 
8. <b>Deal with highly correlated features.</b> Look at correlation plots. Delete or combine highly correlated features.
9. <b>Build a "final" model.</b> Build your first "final" model. Save it for the purpose of future usage.
10. <b>Look at calibration plots.</b> This obviously only true when predicting probabilities but make sure you do this! Even though (almost) all machine learning algorithms have a *predict_proba* function, for most of them it is not really a probability.
11. <b>Interpret the features.</b> Look at the features in your model. Which ones are the most important ones? Which ones lower the target, which ones increase the target? Does this make sense from your intuition? What level of feature understanding is necessary for the problem you try to solve?

Notice that once you start building your own model, you will immediately notice that the points 6. - 11. need to be done iteratively ;-)

### What's next?

You might think now: "Cool, so I know how logistic regression works. But honestly, logistic regression is for losers... Isn't there some cool stuff out there like random forests, neural networks, XGBoost or lightGBM?" I highly recommend <b>not</b> not to jump directly into these but to first familiarize yourself with the steps learned in this notebook. 
- Try to find a model that improves the log loss by e.g. adding new features (I'm 100% sure you can :-))
- Look at the model outputs. Where does the model perform well? Where does it have problems?
- Try building you own model for specific situations (e.g. free kicks, only shots after a corner, ...) See if that improves overall results if you overwrite the current model probabilities with the probabilities of the specified model.
- Make analysis based on the model outputs. Which strikers outperform their expected goals? Which goalie outperforms his expected goals? 

And if you are really interested in lightGBM or XGBoost, you can look into the next notebook where we go through some of the challenges that often come up, when using them ;-)